<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/lib/data_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Datablock API

Notebooks - [datablock](https://github.com/fastai/course-v3/blob/master/nbs/dl2/08_data_block.ipynb), [augmentation](https://github.com/fastai/course-v3/blob/master/nbs/dl2/10_augmentation.ipynb), [text](https://github.com/fastai/course-v3/blob/master/nbs/dl2/12_text.ipynb)

**Todos**
*   Figure out if it is possible to export kaggle_data. Importing .py file gives error
*   Rename extract_col to extract_colval in data bundles in other files eg. Attention Bi-dir LSTM
*   Create a Tuple Dataset or something which passes in the 'y' target as input to the model. So it returns a tuple of (x, y) as the 'x' value and 'y' as the 'y' value
*   Facial Keypoints dataset
*   dfitemlist Extract column takes a post-processing function
*   Rename db.do to db.process in other files which use it
*   Rename extract_col to extract_colval in other files  
*   Mask Itemlist
*   Keypt Itemlist
*   Bounding Box Itemlist
*   Display of Keypt, Bounding Box
*   Augmentation of Keypt, Bounding Box
*   Comments for Paired get item
*   Clean up float_tensor xform for Mask. Also, call it only if img_pair is not NULL. Mask should be converted into tensor in byte_tensor?
*   Cleanup any places with 'WAIT_' and '!!!!!!!'
*   Document concepts of Paired, DL Post Processing in Datablock Overview section
*   Make the non-Dataset and not-Dataloader sections independent of torch/keras framework?
*   Should Plotter be in this file??
*   DONE Comments for Display Data
*   DONE Display Results
*   DONE Separate DataBundle class, DataBundle specs, ItemContainer and ItemList into separate cells




















### Create test CSV file data

In [ ]:
!rm tc.csv
!echo 'reviews','sentiments' >>tc.csv
!echo 'this is my first sentence','happy' >>tc.csv
!echo 'another longer sentence','sad' >>tc.csv
!echo 'this is a text classification set','sad' >>tc.csv
!echo 'fourth line','happy' >>tc.csv
!echo 'fifth line','sad' >>tc.csv
!ls
!cat tc.csv

### Imagenette Data

In [ ]:
from fastai import datasets
from pathlib import Path

IMAGENETTE_160_URL='https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160'
path_imagenette = datasets.untar_data(datasets.URLs.IMAGENETTE_160)
path_imagenette, list(path_imagenette.iterdir()), list((path_imagenette/'val').iterdir())

In [ ]:
from matplotlib import pyplot as plt
import numpy
import PIL

path_tench = path_imagenette/'val'/'n01440764'
file_tench = list(path_tench.iterdir())[0]
img_tench = PIL.Image.open(file_tench)
npa_tench = numpy.array(img_tench)
print (npa_tench.shape)
plt.imshow(img_tench)

### IMDB data

In [ ]:
path_imdb = datasets.untar_data(datasets.URLs.IMDB)
path_imdb, list(path_imdb.iterdir())

### Kaggle Data Download

In [ ]:
def kaggle_data(data_type, data_name):
  # Run this cell and select the kaggle.json file downloaded
  # from the Kaggle account settings page.
  from google.colab import files
  files.upload()

  # Let's make sure the kaggle.json file is present.
  !ls -lha kaggle.json

  # Next, install the Kaggle API client after forcing an upgrade
  !pip uninstall -y kaggle
  !pip install --upgrade pip
  !pip install kaggle==1.5.6
  !kaggle -v

  # Reason for doing a force-upgrade. The underlying problem: Colab installs both py2 and py3 
  # packages, and (for historical reasons) the py2 packages are installed second. kaggle is a 
  # wrapper installed by the kaggle python package; since we do py2 second, the py2 wrapper 
  # is in /usr/local/bin, and happens to be an older version.

  # The Kaggle API client expects this file to be in ~/.kaggle,
  # so move it there.
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/

  # This permissions change avoids a warning on Kaggle tool startup.
  !chmod 600 ~/.kaggle/kaggle.json

  # List available datasets.
  !kaggle {data_type} list

  # First, you have to login to Kaggle, go to that competition's page, navigate to 
  # the Rules tab and accept the terms and conditions. Unless you do that, you will get
  # a 403-Forbidden error when you run the command below

  # Copy the carvana data set locally.
  !kaggle {data_type} download {data_name}

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#export

from torch.utils.data import DataLoader
import IPython.core.debugger as db
from functools import partial
from pathlib import Path
import sys
import pandas as pd
from pandas.api.types import is_categorical_dtype
import torch
from torch import tensor, LongTensor
from torch.utils.data import Sampler, DataLoader, SequentialSampler, RandomSampler
import os
import random
import math
import PIL, mimetypes
from PIL import Image
import spacy, html, re
from spacy.symbols import ORTH
from concurrent.futures import ProcessPoolExecutor
import collections

In [ ]:
from torch import FloatTensor,LongTensor
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
gn_path = 'gdrive/My Drive/Colab Notebooks'
gd_path = 'gdrive/My Drive/Colab Data/fastai-v3'

import sys
sys.path.insert(1, gn_path + '/exp')

In [ ]:
#export

from nb_util import ListContainer, parallel, Counter
# from nb_audio import AudioUtil - this is imported in the AudioItemList after installing torchaudio
from nb_image import ShowImg, PilImg, OcvImg, imgaug_random_resized_crop, imgaug_perspective_warp, imgaug_albu

### Display Data

In [ ]:
#export

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

sns.set(style="darkgrid")

# -----------------------------------------------------
# Generic Plotter class which encapsulates all plotting functionality
# All methods are meant to be statics, so you can create a single
# dummy instance and call them
# -----------------------------------------------------
class Plotter(object):
    def __init__(self, grid_row = 4, grid_col = 4, figsize = 18):
        self.grid_row = grid_row
        self.grid_col = grid_col
        self.figsize = figsize
        self.fig = self._create_fig ()

    # Create a grid of subplots using the given plot type. Each 
    # subplot graphs one feature in the dataframe using the same
    # plot type
    def plot_grid (self, ptype, df, features, target=None, **kwargs):
        # Create a subplot for each feature
        for feature in features:
          if (ptype == 'line'):
            self.create_subplot (ptype, df, y=feature, x=target, **kwargs)
          else:
            self.create_subplot (ptype, df, x=feature, y=target, **kwargs)

    # Create a new subplot at the next position in the grid
    def create_subplot (self, ptype, df, x, y=None, **kwargs):       
        fig = self.fig
        # New subplot will be created at a position that is 1 more 
        # than the existing count of subplots
        pos = len(fig.axes) + 1
        ax = fig.add_subplot(self.grid_row, self.grid_col, pos)

        ax.yaxis.set_major_formatter(ticker.EngFormatter())
        #plt.ticklabel_format(style='plain', axis='y', useOffset=False)

        # Get the plotting function based on the plot type...
        plot_func = self._get_plottype (ptype)
        # ... and call the function
        plot_func (df, x, y, ax=ax, **kwargs)
        return (ax)

    # Create the top-level figure
    def _create_fig (self):
        fig = plt.figure(figsize = (self.figsize, self.figsize))
        # Spacing between the subplots
        plt.subplots_adjust(top=.85, hspace=0.3, wspace=0.3)
        return (fig)
    
    # Get the method for the required plot type
    def _get_plottype (self, ptype):
        # Generate the method name for the plot type
        method_name = '_plot_' + ptype
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, method_name, lambda: "Invalid plot type")
        # Return the method
        return method
 
    # There is a separate method for each type of plot. Each method
    # name follows a naming convention ie. "_plot_<plot type>"
    # Bar graph of the total number of values for each 'x'
    def _plot_count(self, df, x, y, ax):
        ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize=10)
        sns.countplot(x=x, data=df)
 
    # Histogram
    def _plot_dist(self, df, x, y, ax):
        # Trim low and high outliers at both extremes
        trim_df = df[(df[x] > df[x].quantile(0.005)) & (df[x] < df[x].quantile(0.95))]

        sns.distplot(trim_df[x], kde=False)
        #sns.distplot(df[x])
 
    def _plot_line(self, df, x, y, ax, hue=None):
        sns.lineplot(x=x, y=y, hue=hue, data=df)

    # Bar graph plots the mean of all the 'y' values for each 'x'
    def _plot_bar(self, df, x, y):
        sns.barplot(x=x, y=y, data=df)

    def _plot_multicol(self, df, x, y):
        sns.barplot(df.index, df[x])
        
    def _plot_series(self, df, x, y):
        sns.barplot(df.index, df.values)
        
    def _plot_series_horiz(self, df, x, y):
        sns.barplot(df.values, df.index)
        
    def _plot_box(self, df, x, y):
        sns.boxplot(df[x], df[y])
        
    def _plot_reg(self, df, x, y):
        sns.regplot(x=x, y=y, data=df)

    def _plot_scatter(self, df, x, y):
        sns.scatterplot(x=x, y=y, data=df)

In [ ]:
#export

#----------------------------------------------------
# Displays data and results in the appropriate format
# It is able to display data as images, text and tabular format. We will add
# additional formats (eg. audio, video) as we encounter them.
#----------------------------------------------------
class DisplayData():
  # This is not required right now. All the methods are really statics, so an instance
  # need not be initialised. Leaving this for now in case it is required for saving any
  # state.
  def __init__(self):
    pass

  # ----------------------------
  # Display image data, using the Image library
  # ----------------------------
  def display_images(self, x_imgs, y_data, z_data, xyz_procedures, **kwargs):
    sc = ShowImg()
    xyz_methods = self._get_xyz_methods(sc, xyz_procedures)
    sc.show_grid(x_imgs, y_data, z_data, **xyz_methods, **kwargs)

  # ----------------------------
  # Get the actual Image library method names for displaying different variations
  # of image data such as mask, bbox, keypts etc.
  # ----------------------------
  def _get_xyz_methods(self, sc, xyz_procedures):
    xyz_procedures = map(lambda p: 'show_' + p, xyz_procedures) 
    x_procedure, y_procedure, z_procedure = xyz_procedures

    x_method = getattr(sc, x_procedure)
    y_method = getattr(sc, y_procedure)
    z_method = getattr(sc, z_procedure)
    assert((x_method is not None) and (y_method is not None) and (z_method is not None))
    return {'x_method': x_method, 'y_method': y_method, 'z_method': z_method}

  # ----------------------------
  # Display text data as a Pandas Dataframe
  # ----------------------------
  def display_texts(self, x_texts, y_data, z_data, **kwargs):
    from IPython.display import display, HTML
    df_data = {'x': x_texts, 'y': y_data}
    if (z_data): 
      df_data['z'] = z_data
    df = pd.DataFrame(df_data)
    display(HTML(df.to_html(index=False)))

  # ----------------------------
  # Display tabular data as a Pandas Dataframe
  # ----------------------------
  def display_table(self):
    pass

  # ----------------------------
  # Show a Dataframe's metadata summary
  # ----------------------------
  @staticmethod
  def display_summary(mds):
    with pd.option_context(
        'float_format', '{:.2f}'.format,
        'display.max_rows', None
        ):
      display(mds)

  @staticmethod
  def display_rel(fan_df, x, merge_df):
    p = Plotter()
    p.create_subplot ('dist', fan_df, x)

    with pd.option_context(
        'float_format', '{:.2f}'.format,
        ):
      display(merge_df.head(15))

  @staticmethod
  def data_distrib (df, cat_cols, num_cols):
      p = Plotter()
      #Frequency of categorical values
      p.plot_grid ('count', df, cat_cols)

      # Histograms of Numerical features
      p.plot_grid ('dist', df, num_cols)

  # -----------------------------------------------------
  # Bivariate plot of each feature with the target. 
  # If the target is Categorical, use Bar Graphs
  # If the target is Numerical, use Scatter plots for numeric features and..
  #                         ...Box plots for categorical features
  # -----------------------------------------------------
  def data_target (df, cat_cols, num_cols, target):
      p = Plotter()
      if (is_categorical (df, target)):
          plot_pair_target (train_df, cat_cols, target)
          #plot_stackhist(train_df, ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'Sex'], 'Survived')
      else:
          p.plot_grid ('scatter', df, num_cols, target)
          p.plot_grid ('box', df, cat_cols, target)

  # ----------------------------
  # Display audio wave or spectrogram data, using the Audio library
  # ----------------------------
  def display_audio(self, x_data, y_data, z_data, num_cols=8, figsize=None, **kwargs):
    assert(len(x_data) == len(y_data))

    # Compute number of rows and columns in grid
    num_data = len(x_data)
    num_rows = int (math.ceil (num_data / num_cols))

    # Create figure and subplots as per number of rows and columns
    if (figsize is None):
      figsize=(num_cols * 3, num_rows * 3)
    fig,axes = plt.subplots(num_rows, num_cols, figsize=figsize)

    for xd, label, ax in zip (x_data, y_data, axes.flat):
      if (isinstance(xd, tuple)):
        # Data is an audio waveform
        AudioUtil.show_wave(aud=xd, label=label, ax=ax)
      else:
        AudioUtil.show_spectro(spec=xd, label=label, ax=ax)

    for i in range(num_data, len(axes.flat)): axes.flat[i].set_visible(False)

### Datasets

In [ ]:
#export

#----------------------------------------------------
# Dataset containing a multi-dimensional (2D or larger) array
#----------------------------------------------------
class MultiDimDataset():
  def __init__(self, x, y):
    self.x, self.y = x, y
    
  def __len__(self):
    return (self.x.shape[0])
  
  def __getitem__(self, i):
    return (self.x[i], self.y[i])

#----------------------------------------------------
# Dataset containing an ItemList
#----------------------------------------------------
class ILDataset():
  def __init__(self, x, y):
    self.x, self.y = x, y
    
  def __len__(self):
    return (len(self.x))
  
  # ----------------------------
  # Get a (x, y) pair for one sample that can be passed to the model. 
  # ----------------------------
  def __getitem__(self, i):
    return (self.x[i], self.y[i])

  # ----------------------------
  # Get the display data from the the 'x' and 'y' ItemList
  # by converting from the 'obj' format to human-readable.
  # ----------------------------
  def get_display_data(self, i):
    x_obj, y_obj = self[i]
    x_display_data = self.x.get_display_data(x_obj, i)
    y_display_data = self.y.get_display_data(y_obj, i)
    return (x_display_data, y_display_data)

#----------------------------------------------------
# Dataset containing a Paired ItemList as needed for
# Image data with an image + mask, image + bbox, image + key points
# for Segmentation, Object Detection etc.
#----------------------------------------------------
class ILPairedDataset(ILDataset):
  def __getitem__(self, i):
    x_il, y_il = self.x, self.y
    # Get the corresponding 'y' objects for the same index 'i'
    y_objs = y_il[i]

    # Get the 'x' objects after providing the 'x' item list with the paired 'y' objects so
    # that it can process their transforms together
    xy_tuples = x_il.get_paired_items(y_objs, i)

    # If 'i' is a single value, we get back a tuple (x, y) 
    # If 'i' is a slice range, we get back a list of tuples [(x, y), (x, y), ...]
    if (isinstance(xy_tuples, list)):
      # Separate the 'x' values from the 'y' values into two lists
      xs, ys = zip(*xy_tuples)
      x, y = list(xs), list(ys)
    else:
      x, y = xy_tuples
    return (x, y)

#----------------------------------------------------
# Example Dataset for Tabular data. It is not used anywhere at the moment but
# is here as an illustration.
#----------------------------------------------------
class TabDemoDataset(ILDataset):
  # ----------------------------
  # Note that the structure and format of both 'x' and 'y' is entirely up to us. 
  # For instance, they can be lists, numpy arrays, tensors etc of any shape. Pytorch's
  # default_collate function can handle many such formats and can convert them into
  # tensors suitable for feeding to a model. And we can define a custom collate
  # function for formats which are not handled by the default collate.
  #
  # 'x' and 'y' don't have to be a single object eg. 'x' can be a tuple of three
  # objects. This allows us to pass multiple objects to the forward method of
  # the model. For instance, below we could partition the 'x' into a tuple of
  # two objects for categorical variables and continuous variables respectively.
  # The model's forward function would then be 'forward(self, x_cat, x_cont)'
  # ----------------------------
  def __getitem__(self, i):
    return ([self.x[i][:3], self.x[i][3:-1]], self.y[i][-1])

#----------------------------------------------------
# The Fastai Language Model example creates batches in a non-standard way
# See https://github.com/fastai/course-v3/blob/master/nbs/dl2/12_text.ipynb
# Instead of breaking up the contiguous stream into sequential samples, it puts
# them in different batches
#----------------------------------------------------
class FastaiLMDataset(ILDataset):
    def __init__(self, x, y, bs=64, bptt=70, shuffle=False, **kwargs):
        self.x, self.y = x, y
        self.bs,self.bptt,self.shuffle = bs,bptt,shuffle
        total_len = len(x)
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def get_display_data(self, i):
      x_obj, y_obj = self[i]
      x_display_data = self.x.get_display_data(x_obj, i)
      y_display_data = self.x.get_display_data(y_obj, i)
      return (x_display_data, y_display_data)

    def batchify(self):
        #if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = tensor(self.x)
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

def get_dls(train_ds, valid_ds, bs, repro=False, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=not repro, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

### DataLoaders

In [ ]:
#export

#----------------------------------------------------
# Pytorch's Dataloader automatically invokes a Sampler function while preparing a batch to
# figure out which samples should be included in the batch, and in what order. A Sampler
# function returns one mini-batch per iteration, as a list of indexes of samples
# [sample_1_idx, sample_2_idx, ...]
#
# Pytorch then fetches the sample data for those indexes from the Dataset and 
# calls a Collate function with the list of samples in one mini-batch. Each sample
# is a sub-list (or tuple) of two elements, for the 'x' and 'y' data.
# [[sample_1_x, sample_1_y], [sample_2_x, sample_2_y], ...] OR
# [(sample_1_x, sample_1_y), (sample_2_x, sample_2_y), ...]
#
# The Collate function joins all the 'x' samples into a single 'x' tensor, and 
# the 'y' samples into a single 'y' tensor
#
# Pytorch has default Sampler and Collate functions, or we can write custom functions
#
# During text classification, since texts can be of different lengths, the samplers create mini-batches
# with texts of similar lengths to make it efficient, so that the model doesn't have to handle
# texts of vastly varying lengths in the same batch.
#
# Then the collate function pads those texts to make them the same length so that they can be
# converted into a tensor with a rectangular shape
#----------------------------------------------------

#----------------------------------------------------
# Sort samples by text length, used for validation data
#----------------------------------------------------
class SortSampler(Sampler):
    def __init__(self, data_source, key): 
      self.data_source,self.key = data_source,key

    def __len__(self): return len(self.data_source)
    def __iter__(self):
      # Sort the list using the key function
      return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

#----------------------------------------------------
# For training data, we want some kind of randomness in addition to sorting by text length. 
# So first, we shuffle the texts and build megabatches of size 50 * bs. We sort those 
# megabatches by length before splitting them into 50 minibatches. That way we have randomized 
# batches of roughly the same length.
#
# Then we make sure to have the biggest batch first and shuffle the order of the other 
# batches. We also make sure the last batch stays at the end because its size is probably 
# lower than batch size.
#----------------------------------------------------
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        #idxs = torch.randperm(len(self.data_source))
        idxs = list(range(len(self.data_source)))
        random.shuffle(idxs)

        # Create megabatches
        megabatch_len = self.bs * 50
        megabatches = [idxs[i:i+megabatch_len] for i in range(0, len(self.data_source), megabatch_len)]

        # Sort within a megabatch and then flatten all megabatches into a single list
        sorted_megabatches = [sorted(b, key=self.key, reverse=True) for b in megabatches]
        sorted_idxs = [j for i in sorted_megabatches for j in i]

        # Create mini-batches
        batches = [sorted_idxs[i:i+self.bs] for i in range(0, len(sorted_idxs), self.bs)]

        # Move the mini-batch with the longest length to the top
        max_idx = max([ck[0] for ck in batches], key=self.key)
        max_idx = max(list(range(len(batches))), key=lambda x: self.key(batches[x][0]))
        batches[0],batches[max_idx] = batches[max_idx],batches[0]

        mid_batch_idxs = list(range(1, len(batches)-1))
        random.shuffle(mid_batch_idxs)
        shuffled_mid_batch = [batches[i] for i in mid_batch_idxs]

        sorted_idxs = batches[0] + [item for sublist in shuffled_mid_batch for item in sublist] + batches[-1]
        return iter(sorted_idxs)

#----------------------------------------------------
# Collate function for image data
# It gets a list of (x, y) tuples
# NB: This is not used right now. It depends on the 'xs' and 'ys' being tensors.
#----------------------------------------------------
def collate(b):
  # 'xs' and 'ys' are a list of tensors
  xs,ys = zip(*b)

  # Join the list of 'x' into one tensor, and similarly for the 'y'
  return torch.stack(xs),torch.stack(ys)

#----------------------------------------------------
# Collate function for text classification data
# Add the 'pad_idx' token at the end of each sequence to make them 
# all the same size when batching them. 
#----------------------------------------------------
def pad_collate(samples, pad_idx=1, pad_first=False):
  # Get the length of the longest sentence in the batch
  max_len = max([len(s[0]) for s in samples])

  # Create empty 'x' result tensor filled with the 'pad_idx'
  res = torch.zeros(len(samples), max_len).long() + pad_idx

  # Go through each sample and fill its data into the result tensor
  for i,s in enumerate(samples):
    if pad_first: res[i, -len(s[0]):] = LongTensor(s[0])
    else:         res[i, :len(s[0]) ] = LongTensor(s[0])

  # Return 'x' result tensor, and 'y' data
  return res, tensor([s[1] for s in samples])


### DataBundle

In [ ]:
#----------------------------------------------------
# There are three main logical entities for data preparation:
#   ItemContainer - the original source of the data, from which an ItemList is loaded.
#                   eg. Folder, CSV File, Dataframe, Json File
#   ItemList - represents a list of 'rows' or 'items' of a certain type of data format. Operations include splitting it row-wise into subsets and converting from one type to another. 
#              This is the core data structure that is processed throughout the data preparation pipeline.
#              eg. File Paths, Image Files, Audio Files, Text Files, Class Names, Class Ids, Multi Class Names, Sentences, Words, Word Ids
#              eg. Chars, Docs, Bounding Box, Image Point, Segmentation Mask
#   DataBundle - represents a data preparation flow for an application use case. It does all the orchestration.
#                eg. Image Classification, Text Classification, Language Model, Multi Image Classification, Object Detection, Object Segmentation
#
# Once loaded, an ItemList goes through one split and a series of data type conversions (viz. extractions, converters and transforms) till we end up with 
# data in the format required for model training. Each of these takes an ItemList and outputs another ItemList, but the semantics are slightly different:
#   Split - split the items row-wise 
#   Extract - take the 'raw loaded rows' and extract the minimal 'columns' needed for the ML model viz. 'x' feature items and 'y' label items
#   Convert - enrich the 'x' and 'y' items (eg. fill missing values) or convert them from one data type to another
#   Transform - as opposed to Convert which is a 'permanent one-time' conversion, this is a 'dynamic run-time' conversion where the converted output is 
#               processed for model training but not persisted.
#
# The end outcome is to create a DataBundle with three Datasets (for training, validation and test), each with a 'x' and 'y' tensor of floats
#   and three Dataloaders which wrap the corresponding Dataset, which can be used for model training
#
# The end-to-end flow to prepare data for training is:
#   Load data from an ItemContainer which is a 'logical container' for your data. It returns an ItemList which contains a list of all the 'logical rows' of your data.
#      An ItemContainer has subclasses of different types (eg. a Folder Container which contains data rows as files, or a CSV Container which contains data rows)
#      An ItemList has subclasses of different types (eg. an Image File item list holds image files, a Sentence item list holds text sentences)
#      Each 'logical row' of data is known as an 'item'
#   Split all the data rows from the ItemList into three separate ItemLists for training, validation and test data respectively
#      There are various procedures to do this split - based on a containing folder for file-based lists, randomly based on a percentage, based on a custom function etc
#   Extract the 'x' items from each of the three ItemLists
#      There are various procedures to do the extraction - based on a column for Dataframe lists, based on a parent folder for file-based lists, by reading the text for a text-file list etc
#   Convert the extracted 'x' items via a sequential pipeline of conversion steps. Each conversion results in a target ItemList of the same or different type than the source
#      There are various procedures which know how to do a specific type of conversion (eg. from a text sentence to a list of words)
#   Similarly, extract the 'y' items from each of the three ItemLists and then convert them via another pipeline
#   Create three Datasets from each of the three pairs of 'x' and 'y' ItemLists, passing it any custom parameters if needed
#   Then create DataLoaders from those Datasets, passing it any custom sampler and/or collate functions if needed. Note that we may need
#      to use different parameter settings for the training and validation DataLoaders
#   During training, optionally transform the 'x' items dynamically by performing additional conversions via a sequential pipeline.
#      Since these conversions are performed at 'runtime' the converted data is not persisted but used for training immediately
#      eg. Images can be loaded from image files and augmented, resized, cropped etc. They are then converted to pixels and eventually to tensors.
#          Doing this pre-training and storing the transformed data would be too memory-intensive.
#          So the image data is stored as image filenames and transformed batch-wise during training
#   
#   Each step above is intended to be composed in different ways to allow you to flexibly create an end-to-end pipeline for different application use cases.
#   A DataBundle processes this whole pipeline and lets you specify the pipeline declaratively.
#      A DataBundle has many subclasses for each of the common use cases, which know how to build a pipeline for that use case.
#      eg. an Image Classification data bundle, Language Model, Object Detection etc
#----------------------------------------------------

In [ ]:
#export

#----------------------------------------------------
# Defines and orchestrates an end-to-end data preparation pipeline for different use cases
# The pipeline is defined via several declarative parameters for each step (ie. Load, Split, Extract, Convert and Transform)
#----------------------------------------------------
class DataBundle:
  def __init__(self, load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=None, ds_params=None, dl_params=None):
    self.all_rows = None
    self.load_params = load_params
    self.split_params = split_params
    self.extract_x_params, self.extract_y_params = extract_x_params, extract_y_params
    self.convert_x_params, self.convert_y_params = convert_x_params, convert_y_params
    self.xform_x_params = xform_x_params
    self.ds_params = ds_params if (ds_params is not None) else {'target_ds': ILDataset}
    self.dl_params = dl_params 
    self.train_rows, self.valid_rows, self.test_rows = None, None, None

  # ----------------------------
  # !!!!!!!! TODO get rid of this once all callers have been renamed
  # ----------------------------
  def do(self):
    return self.process()

  # ----------------------------
  # Orchestrates and data preparation pipeline using the specified parameters for each step
  # ----------------------------
  def process(self, steps=['load', 'post_load']):
    if ('load' in steps):
      # Create a Container and Load all the data rows into an ItemList
      self.all_rows = self._load (**self.load_params)
      if (steps == ['load']): return self.all_rows

    if ('post_load' in steps):
      # Continue with post-loading split, extract and convert
      assert(self.all_rows)
      self._post_load()

  # ----------------------------
  # Split, extract and convert the loaded data
  # ----------------------------
  def _post_load(self):
    # Split the the data rows into three subset ItemLists for training, validation and test
    self._split(**self.split_params)

    # Prepare three Datasets by extracting 'x' and 'y' data items and going through a sequence of conversion steps to produce the required ItemList data type
    ds_list = self._rows_to_ds(self.extract_x_params, self.extract_y_params, self.convert_x_params, self.convert_y_params, self.xform_x_params, self.ds_params)
    
    # Free up memory
    del self.all_rows, self.train_rows, self.valid_rows, self.test_rows
    self.all_rows, self.train_rows, self.valid_rows, self.test_rows = None, None, None, None

    self._print_ds(ds_list)

    # Prepare three Dataloaders from the Datasets
    dl_list = self._ds_to_dl(ds_list, self.dl_params)
    
    assert((len(ds_list) == len(dl_list)) and (0 < len(ds_list) <= 3))

    (self.train_ds, _) = ds_list[0]
    self.train_dl = dl_list[0]

    if (len(ds_list) > 1): 
      (self.valid_ds, _) = ds_list[1]
      self.valid_dl = dl_list[1]
    else:
      self.valid_ds, self.valid_dl = None, None

    if (len(ds_list) > 2):
      (self.test_ds, _) = ds_list[2]
      self.test_dl = dl_list[2]
    else:
      self.test_ds, self.test_dl = None, None

  # ----------------------------
  # Load the data from the source Container into the target ItemList
  # ----------------------------
  def _load(self, source, target_cls, **kwargs):
    # Create a Container of subclass type 'source' and load the data
    cntnr = source(**kwargs)

    # The Container creates an ItemList of subclass type 'target_cls'
    all_rows = cntnr.load(target_cls)
    
    print ("%s loaded %s items of type %s" % (cntnr.__class__.__name__, len(all_rows), all_rows.__class__.__name__))
    return (all_rows)

  # ----------------------------
  # Split the loaded data based on a given 'split_procedure' and any arguments required by that
  # split procedure
  # ----------------------------
  def _split(self, split_procedure, **kwargs):
    if (split_procedure is None):
      print ('No Split')
      self.train_rows, self.valid_rows, self.test_rows = self.all_rows, None, None
      return

    # Map the given 'split_procedure' to the corresponding method on the
    # loaded ItemList
    split_method = getattr(self.all_rows, split_procedure)

    # Call the split method and pass it any additional arguments 
    # The loaded data is split into three separate ItemLists
    self.train_rows, self.valid_rows, self.test_rows = split_method(**kwargs)
    print ("Split using %s into %s, %s and %s items of type %s" % (split_procedure, len(self.train_rows), len(self.valid_rows), len(self.test_rows) if self.test_rows is not None else 0, self.train_rows.__class__.__name__))

  # ----------------------------
  # Prepare three Datasets from the training, validation and test data rows
  # First extract the 'x' and 'y' items and then convert each one to the required
  # type of ItemList
  # ----------------------------
  def _rows_to_ds(self, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params, ds_params):
    # Initialise list for three Datasets
    ds_list = []

    # We loop through the training, validation and test data and execute the
    # extraction and conversion pipeline for each
    #
    # We maintain a Boolean flag indicating whether we're processing training data 
    # or not (ie. validation/test data), which will be used by the converters.
    rows_list = [(self.train_rows, True)]
    if (self.valid_rows is not None): rows_list.append((self.valid_rows, False))
    if (self.test_rows is not None): rows_list.append((self.test_rows, False))
    #
    # Some converters may need to save state while processing the training data
    # and then reuse that state while processing validation or test data. For instance
    # it may create a vocab using the training data and use that same vocab for
    # converting the validation and test data
    #
    # We initialise a dictionary to hold Conversion State and pass it to each converter.
    # Any converter can then store its state under its own key in this dictionary. That
    # state is then available to it while processing validation and test data
    self.convert_state_x, self.convert_state_y = {}, {}
 
    # Go through three iterations for the training, validation and test data
    for rows, in_train in rows_list:
      # Extract the 'x' and 'y' items
      x_il = self._extract(rows, **extract_x_params)
      y_il = self._extract(rows, **extract_y_params)

      # Do all the conversion steps for the 'x' items. The output of one step
      # becomes the input for the subsequent step
      for converter in convert_x_params:
        x_il = x_il.convert(self.convert_state_x, in_train, **converter)

      # Do all the conversion steps for the 'y' items.
      for converter in convert_y_params:
        y_il = y_il.convert(self.convert_state_y, in_train, **converter)

      # Set the transforms to the 'x' ItemList. They are not executed now, but
      # will be executed dynamically during model training
      if (xform_x_params is not None):
        x_il.add_xform(xform_x_params)

      # Create a Dataset using the 'x' and 'y' ItemLists
      ds = self._create_ds(x_il, y_il, **ds_params)

      # Add the Dataset to the list of Datasets, along with the corresponding
      # in_train flag
      ds_list.append((ds, in_train))

    return (ds_list)
  
  # ----------------------------
  # Extract items (either 'x' or 'y') from the given data rows
  # ----------------------------
  def _extract(self, rows_il, extract_procedure, target_cls, **kwargs):
    if (extract_procedure is None):
      print ("No extraction")
      return (rows_il)

    # Get the extract method on the ItemList and invoke it. It returns a Python list of extracted items
    # Create a new target_cls ItemList using the extracted Python list of items
    extract_method = getattr(rows_il, extract_procedure)
    assert(extract_method is not None)

    # Pass in an empty dictionary which the 'extract_method' can fill with extra arguments
    # that it wants to pass back. Those extra arguments are then available to the constructor
    # of the new target ItemList
    kwargs['extra_args'] = {}
    extract_list = extract_method(**kwargs)

    # If the 'target_cls' is of the same type, then clone the data rows with the extracted
    # list and any extra arguments
    if (rows_il.__class__ == target_cls):
      extract_il = rows_il.clone(extract_list, **kwargs)
    else:
      extract_il = target_cls(extract_list, **kwargs)

    print ("Extracted %s items of type %s using %s" % (len(extract_il), extract_il.__class__.__name__, extract_procedure))
    return (extract_il)

  #-------------------------
  # Create a Dataset using the given parameters
  # ----------------------------
  def _create_ds(self, x_il, y_il, target_ds, **kwargs):
    assert (target_ds is not None)
    ds = target_ds(x_il, y_il, **kwargs)
    return ds

  # ----------------------------
  # Prepare a list of DataLoaders from the list of Datasets
  # ----------------------------
  def _ds_to_dl(self, ds_list, dl_params):
    # dl_params is a tuple (params for training dl, params for valid/test dl)
    if (dl_params is None):
      # If no params are passed in, use an empty param list to create a DataLoader with defaults 
      dl_params=({}, {})
    train_dl_params, valid_dl_params = dl_params

    # Create a DataLoader for each Dataset in the list
    dl_list = []
    for ds, in_train in ds_list:
      # Use the training dl params or the valid/test dl params
      params = train_dl_params if in_train else valid_dl_params
      dl = self._create_dl(ds, in_train, **params)
      dl_list.append(dl)

      # Do any post-processing steps if required
      post_proc_params = getattr(self, 'post_proc_params', None)
      if (post_proc_params is not None):
        for post_proc_param in post_proc_params:
          self._post_proc(dl, ds, in_train, **post_proc_param)

    return dl_list

  # ----------------------------
  # Do a data post-processing step using one batch of data that we fetch 
  # from the DataLoader
  #
  # NB: Right now the only use case for this is to do normalisation for image datasets
  # by calculating the mean and std on a batch of data. So below, we make the assumption
  # that we should fetch only one batch of data, do post processing on only the 'x' 
  # ItemList and don't get any return values from the post-processing step. If we
  # find any other use case where those assumptions are not valid, we will have to
  # change the logic below as required.
  # ----------------------------
  def _post_proc(self, dl, ds, in_train, proc_procedure, **kwargs):
    # Fetch a batch of data from the Dataloader
    xb, yb = next(iter(dl))
    # Get the x and y ItemLists
    x_il, y_il = ds.x, ds.y

    # Call the post-processing step. If this is a training dataset, the method may
    # add arguments into the 'kwargs', so that they can be passed to the validation
    # datasets.
    proc_method = getattr(x_il, proc_procedure)
    assert(proc_method is not None)
    proc_method(xb, in_train, **kwargs)

  #-------------------------
  # Create a DataLoader from a Dataset using the given parameters
  # ----------------------------
  def _create_dl(self, ds, in_train, bs=64, shuffle=False, sampler_fn=None, collate_fn=None, **kwargs):
    sampler = self.get_sampler(ds, in_train, bs, sampler_fn, **kwargs)
    dl = DataLoader(ds, batch_size=bs, shuffle=shuffle, sampler=sampler, collate_fn=collate_fn)
    return dl

  # ----------------------------
  # Get a Sampler for the DataLoader
  # We make this a separate method to allow subclass DataBundles to override with a custom method
  # ----------------------------
  def get_sampler(self, ds, in_train, bs, sampler_fn, **kwargs):
    sampler = sampler_fn(ds) if sampler_fn is not None else None
    return sampler

  # ----------------------------
  # Temporary utility to print the contents of a Dataset
  # ----------------------------
  def _print_ds(self, ds_list):
    for ds, _ in ds_list:
      print ('Final', ds.x, '\n', ds.y)

  # ----------------------------
  # Display a batch of data from the given Dataset. Use the given indexes for the
  # data rows to display, or pick them randomly
  # ----------------------------
  def display_batch(self, ds_type='train', idxs=None, **kwargs):
    # Get the Dataset
    ds = getattr(self, f'{ds_type}_ds')

    # Pick random indexes if none are provided
    if (idxs is None):
      all_items = range(len(ds))
      num_batch = 16
      idxs = random.sample(all_items, num_batch)

    # Get the display data for all the index rows
    # We get back a list of (x, y) tuples - [(x1, y1), (x2, y2),...] which we
    # convert into separate 'x' and 'y' lists - [x1, x2, ...] and [y1, y2, ...]
    xy_tuples = [ds.get_display_data(i) for i in idxs]
    x_display_data, y_display_data = zip(*xy_tuples)

    # !!!!!!!! REMOVE
    #x_display_data, y_display_data = list(x_display_data), list(y_display_data)
    #x_display_data = [ds.x.get_display_data(i) for i in idxs]
    #y_display_data = [ds.y.get_display_data(i) for i in idxs]

    # Pass the data for display along with any display arguments. Any arguments
    # passed in override the pre-define display arguments.
    kwargs = {**self.display_params, **kwargs}
    self._display(x_display_data, y_display_data, z_display_data=None, **kwargs)

  # ----------------------------
  # Use the 'DisplayData' class to display (x_feature, y_target, z_predicted) data. This data
  # could be training input data (in which case there is no 'z_predicted'), or result data.
  # ----------------------------
  def _display(self, x_display_data, y_display_data, z_display_data, layout_procedure, xyz_procedures=None, **kwargs):

    # DisplayData supports a variety of 'layouts' that are appropriate for different data formats such as images, text
    # and tabular data
    sd = DisplayData()
    layout_method = getattr(sd, layout_procedure)
    assert(layout_method is not None)

    if (xyz_procedures == 'custom'):
      # This is how we handle custom situations, but hopefully we are not going to need
      # this. If we do need it, we can decide whether 'xyz_procedures' is the string 'custom'
      # or whether it is still a tuple like ('custom', 'image', 'custom') which might be overkill
      pass
    elif (xyz_procedures is not None):
      # 'xyz_procedures' are used by image data to display different types of image-related
      # artifacts such as images, segmentation masks, bounding boxes and key points.
      assert(isinstance(xyz_procedures, tuple) and (len(xyz_procedures) == 3))
      layout_method(x_display_data, y_display_data, z_display_data, xyz_procedures, **kwargs)
    else:
      # Display the data
      layout_method(x_display_data, y_display_data, z_display_data, **kwargs)

  # ----------------------------
  # Display a batch of results from the given (x, y, z) data. This is model data
  # in tensor format rather than ItemLists, so convert it into human-displayable
  # format. Use the given indexes for the rows to display, or pick them randomly.
  # ----------------------------
  def display_results(self, x_objs, y_objs, pred_objs, idxs=None, **kwargs):
    # Pick random indexes if none are provided
    if (idxs is None):
      all_items = range(len(x_objs))
      num_batch = 8
      idxs = random.sample(all_items, num_batch)

    # We just need the Dataset class, not the instance, so that the appropriate methods 
    # for the type of data are called. So either train or valid Dataset is fine.
    ds = self.train_ds

    # Convert the data from model-readable object format to human-displayable
    # format. We get back three separate 'x', 'y' and 'z' lists - [x1, x2, ...],
    # [y1, y2, ...] and [z1, z2, ...]
    x_display_data = [ds.x.get_display_data(x_objs[i], 0) for i in idxs]
    y_display_data = [ds.y.get_display_data(y_objs[i], 0) for i in idxs]
    pred_display_data = [ds.y.get_display_data(pred_objs[i], 0) for i in idxs]

    # Pass the data for display along with any display arguments. Any arguments
    # passed in override the pre-define display arguments.
    kwargs = {**self.display_params, **kwargs}
    self._display(x_display_data, y_display_data, pred_display_data, **kwargs)

### Item Container

In [ ]:
#export

#----------------------------------------------------
# Base class for a Container which loads data rows
#----------------------------------------------------
class ItemContainer:
  def __init__(self):
    pass

  def load(self, target_cls):
    print ('ItemContainer load not implemented')

#----------------------------------------------------
# A Folder is a Container where each file under the root 'folder_path' represents an individual
# data row. The list of files can be filtered by selecting only specific sub-folders directly under the
# folder_path (ie. one level under it) and specific file extensions.
#----------------------------------------------------
class FolderItemContainer(ItemContainer):
  def __init__(self, folder_path, include_subfolders=None, recursive=True, extensions=None):
    self.folder_path = Path(folder_path)
    if (include_subfolders is not None):
      # Make a set of the included sub folders one level under the folder_path
      self.include_subfolders = set ([self.folder_path / sub_folder for sub_folder in include_subfolders])
    else:
      self.include_subfolders = None
    self.recursive = recursive
    self.extensions = extensions
    super().__init__()

  # ----------------------------
  # Return an ItemList of type 'target_cls' using a list of all file names under the 
  # folder_path that satisfy the filter criteria
  # NB: Right now the recursive flag is not used and defaults to True
  # ----------------------------
  def load(self, target_cls):
    # Read all files recursively under the folder
    file_list = []
    self._get_files(self.folder_path, file_list)

    # Create the Item List with the list of files
    all_rows = target_cls(file_list, self.folder_path)
    return all_rows

  # ----------------------------
  # Recursive function that reads all files under the folder_path and adds their paths
  # to the 'file_list'
  # ----------------------------
  def _get_files(self, folder_path, file_list):
    # NB: Use the folder_path argument that is passed in recursively. Do not use self.folder_path

    # Loop through all files in the folder_path
    scan_list = os.scandir(folder_path) 
    for o in scan_list:

      if (o.is_dir() and self._is_included(o)):
        # Recursively process any sub-directories which are included based on the filter criteria
        self._get_files(o.path, file_list)
      elif (o.is_file()):
        # Add any files that have the required extension
        op = Path(o.path)
        if ((self.extensions is None) or (op.suffix.lower() in self.extensions)):
          file_list.append(op)

    scan_list.close()

  # ----------------------------
  # Return True if the current 'dir_obj' directory meets the 'include_subfolders' filter
  # criteria.
  #
  # If a list of 'include_subfolders' is defined, check whether the current directory 
  # or any of its ancestors is in that list
  # ----------------------------
  def _is_included(self, dir_obj):
    assert(dir_obj.is_dir())

    # Return True if no criteria is defined
    if (self.include_subfolders is None):
      return True
    
    # Check if any of the ancestors of the current directory overlaps with
    # the list of included sub folders.
    dir_p = Path(dir_obj.path)
    # Set of current directory and its ancestors
    dir_parents = set(list(dir_p.parents) + [dir_p])
    if (bool(self.include_subfolders & dir_parents)):
      return True

    return False

#----------------------------------------------------
# A CSV file is a Container where each line in the file represents an individual
# data row. In the simple case, only a 'csv_path' is provided which it loads into
# a Pandas DfItemList.
#
# In the advanced case, it loads a TabularItemList. Here, we could optionally be 
# given a test CSV file, additional CSV files with related data rows, and 
# a 'prepare_fn' to do any data cleanup after loading.
#----------------------------------------------------
class CSVItemContainer(ItemContainer):
  def __init__(self, csv_path, test_csv_path=None, related_csv_paths=None, prepare_fn=None):
    # Primary CSV file
    self.csv_path = csv_path

    # CSV file for test data
    self.test_csv_path = test_csv_path

    # CSV files for related data
    assert((related_csv_paths is None) or isinstance(related_csv_paths, list))
    self.related_csv_paths = related_csv_paths

    # Data preparation function
    self.prepare_fn = prepare_fn

    super().__init__()

  # ----------------------------
  # Return a DfItemList by using Pandas to read the CSV file into a Dataframe
  # ----------------------------
  def _load_csv(self, target_cls, is_test=False):
    # Load related CSV data (for Tabular only)
    if (self.related_csv_paths):
      related_dfs = {rp.stem: target_cls.open_csv(rp) for rp in self.related_csv_paths}
    else:
      related_dfs = None

    # Load the CSV data and create the ItemList
    csv_file = self.csv_path if not is_test else self.test_csv_path
    df = target_cls.open_csv(csv_file)
    rows = target_cls(items=None, df=df, related_dfs=related_dfs)

    # Data preparation (for Tabular only)
    if (self.prepare_fn):
      rows = self.prepare_fn(rows, is_test=is_test)

    return rows

  # ----------------------------
  # Return a DfItemList by using Pandas to read the CSV file into a Dataframe
  # ----------------------------
  def load(self, target_cls):
    assert(target_cls in [DfItemList, TabularItemList])

    # Load primary data (For Tabular, also load related data and run data preparation)
    all_rows = self._load_csv(target_cls)

    # Test CSV file goes through the identical steps to the primary CSV file including
    # the same related dfs and data preparation
    if (self.test_csv_path):
      test_rows = self._load_csv(target_cls, is_test=True)
      # Save the test data with the primary ItemList object
      all_rows.add_test_rows(test_rows)

    return all_rows

### ItemList

In [ ]:
#export

#----------------------------------------------------
# Base class for an ItemList which holds a list of items of a certain data format
# Operations on the ItemList are: Split, Extract, Convert and Transform
# Transforms are applied dynamically when you get an item from the ItemList
#----------------------------------------------------
class ItemList(ListContainer):
  def __init__(self, items, **kwargs):
    # Used when we split one ItemList into three subsets. The '_copy_new' list contains all
    # attributes of the source ItemList which need to be copied as initialisation arguments 
    # into each of the subset ItemLists
    self._copy_new = []

    self.xforms = None
    super().__init__(items)

  # ----------------------------
  # Split the data sequentially (as opposed to randomly) based on a percentage ratio
  # ----------------------------
  def split_sequential(self, train_ratio, valid_ratio, test_ratio=0.0):
    return self._split_ratio(train_ratio, valid_ratio, test_ratio, randomly=False)

  # ----------------------------
  # Split randomly into three buckets based on a percentage ratio
  # ----------------------------
  def split_random(self, train_ratio, valid_ratio, test_ratio=0.0):
    return self._split_ratio(train_ratio, valid_ratio, test_ratio, randomly=True)

  # ----------------------------
  # Internal function to split the data based on a percentage ratio. We can put
  # data into each bucket either randomly or sequentially
  # ----------------------------
  def _split_ratio(self, train_ratio, valid_ratio, test_ratio=0.0, randomly=True):
    # We are allowed to choose ratios that don't add up to 100%
    assert(train_ratio + valid_ratio + test_ratio <= 1.0)

    # Make a list of row indexes
    num_idxs = len(self)
    idxs = list(range(num_idxs))

    # Shuffle them randomly if required. Otherwise they are in sequential order
    if (randomly):
      random.shuffle(idxs)
  
    # Use the percentage ratio to calculate the number of rows going into each bucket.
    # Take the first 'n' indexes for training, the next 'm' for validation
    # and the next 'p' for test
    train_num_idxs = math.ceil(num_idxs * train_ratio)
    valid_num_idxs = math.floor(num_idxs * valid_ratio)
    assert(train_num_idxs + valid_num_idxs <= num_idxs)
    train_idxs = idxs[:train_num_idxs]
    valid_idxs = idxs[train_num_idxs:train_num_idxs + valid_num_idxs]
    test_idxs = None
    if (test_ratio > 0.0):
      test_num_idxs = math.floor(num_idxs * test_ratio)
      assert(train_num_idxs + valid_num_idxs + test_num_idxs <= num_idxs)
      test_idxs = idxs[train_num_idxs + valid_num_idxs:train_num_idxs + valid_num_idxs + test_num_idxs]

    # Get the ItemLists of rows corresponding to the indexes in each bucket
    train_rows, valid_rows, test_rows = self.split_idxs(train_idxs=train_idxs, valid_idxs=valid_idxs, test_idxs=test_idxs)

    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Split based on a given list of indexes for each bucket
  # ----------------------------
  def split_idxs(self, train_idxs, valid_idxs, test_idxs=None):
    # Get the list of items based on the indexes in each bucket
    
    # !!!!!!!!!!! See if we can get this to work and get rid of the 
    # function _get_item_for_split_wo_xform
    #train_items = self.items[train_idxs]
    #valid_items = self.items[valid_idxs]
    #test_items = self.items[test_idxs] if (test_idxs is not None) else None

    train_items = self._get_item_for_split_wo_xform(train_idxs)
    valid_items = self._get_item_for_split_wo_xform(valid_idxs)
    test_items = self._get_item_for_split_wo_xform(test_idxs) if (test_idxs is not None) else None

    # Get the ItemLists of rows corresponding to the items in each bucket
    train_rows, valid_rows, test_rows = self.split_list(train_items, valid_items, test_items)

    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Split based on a given Python list of items for each bucket
  # ----------------------------
  def split_list(self, train_item_list, valid_item_list, test_item_list=None):
     # Clone myself (viz. the source ItemList) to get three ItemLists using the
    # list of items in each bucket
    train_rows = self.clone(train_item_list)
    valid_rows = self.clone(valid_item_list)
    test_rows = self.clone(test_item_list)

    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Split based on a custom provided function
  # ----------------------------
  def split_custom(self, custom_fn, **kwargs):
    print ('ItemList split_custom', custom_fn)
    train_rows, valid_rows, test_rows = custom_fn(**kwargs)
    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Split based on a folder path - only applies to file-path-based ItemLists
  # ----------------------------
  def split_path(self, train_folder, valid_folder, test_folder=None):
    print ('ItemList split_path not implemented')

  # ----------------------------
  # Set the list of dynamic transforms
  # ----------------------------
  def add_xform(self, xform_x_params):
    self.xforms = xform_x_params

  # ----------------------------
  # Make a clone ItemList with the same initialisation arguments as myself but based on
  # the given Python list of items. Optionally, we can be given some extra arguments to
  # be used for initialisation. ItemLists that store data in backend systems rather than
  # in the list of 'items' could use those extra arguments for initialisation, instead 
  # of or in addition to the items themselves.
  # ----------------------------
  def clone(self, items, extra_args={}, **kwargs):
    # To prepare the clone, we need either a new set of items, or some extra arguments.
    # If neither are given, there is nothing to do.
    if (not items and not extra_args):
      return (None)

    # Keyword arguments for initialisation using the attributes to be copied over 
    # to the cloned ItemList
    copy_args = {k:getattr(self, k, None) for k in self._copy_new}
    # Add the list of items to the keyword arguments, along with the copied attributes.
    # The extra arguments are merged last as they should override the copies attributes.
    kwargs = {'items': items, **copy_args, **extra_args}
    # Now create a new instance of my ItemList type using the keyword arguments
    il = self.__class__(**kwargs)

    return (il)

  # ----------------------------
  # Make a clone ItemList with the same initialisation arguments as myself but based on
  # the given Python list of items 
  # !!!!!!!!!!!
  # ----------------------------
  def OBSOLETE_new(self, items):
    if ((items is None) or len(items) == 0):
      return (None)
    
    # Keyword arguments for initialisation using the attributes to be copied over 
    # to the cloned ItemList
    copy_args = {k:getattr(self, k, None) for k in self._copy_new}
    # Add the list of items to the keyword arguments
    kwargs = {'items': items, **copy_args}
    # Now create a new instance of my ItemList type using the keyword arguments
    il = self.__class__(**kwargs)

    return (il)

  # ----------------------------
  # Use 'convert_procedure' to convert myself to a 'target_cls' ItemList
  # The 'in_train' flag is true if this is training data. 'convert_state' dict can be
  # used to save state generated while processing training data, so that we can refer to
  # it later while processing validation or test data.
  # ----------------------------
  def convert(self, convert_state, in_train, target_cls, convert_procedure, **kwargs):
    if (convert_procedure == 'Custom'):
      # Use a custom converter
      convert_method = self.convert_custom
    else:
      # Get the convert method
      convert_method = getattr(self, convert_procedure, None)
      assert(convert_method is not None)


    # Invoke the convert method on all items to get a Python list of target items
    items = self.items[:]
    target_items = convert_method(items=items, convert_state=convert_state, in_train=in_train, **kwargs)

    if (target_cls != self.__class__):
      # Create a new 'target_cls' ItemList using the target items
      target_kwargs = {**kwargs, **convert_state}
      target_il = target_cls(target_items, **target_kwargs)
      assert(target_il.__class__ == target_cls)
    else:
      # If the target class is the same as our ItemList's existing class, we assume
      # that the convert method has completed the conversion, and there is nothing
      # further to do. Just return ourselves.
      target_il = self
        
    print ("Converted %s items to type %s using %s" % (len(target_il), target_il.__class__.__name__, convert_procedure))
    return (target_il)
 
  # ----------------------------
  # Convert items using the given 'custom_fn'
  # ----------------------------
  def convert_custom(self, items, convert_state, in_train, custom_fn, **kwargs):
    print ('ItemList convert_custom')

  # ----------------------------
  # Extract items (either 'x' or 'y') using the given 'custom_fn'
  # ----------------------------
  def extract_custom(self, custom_fn, **kwargs):
    # Apply the custom function to every item to produce a Python list
    y_list = list(map(custom_fn, self[:]))
    return (y_list)

  # ----------------------------
  # Extract items using the a given column
  # Implemented in the subclass
  # !!!!!! 'col' should be a list not a single column
  # ----------------------------
  def extract_colval(self, col):
    y_list = None
    print ('ItemList extract_colval not implemented')
    return (y_list)

  # ----------------------------
  # Some ItemLists may store the 'real' data in another backend system eg. a Dataframe
  # or a database. Those ItemLists would then override this method to map the 'item' in 
  # the ItemList to the real data 'object'. For all other ItemLists the default behaviour 
  # is that the data object is the same as the item.
  # ----------------------------
  def _get_object(self, item): return item

  # ----------------------------
  # Transform items dynamically using the given 'xform_procedure'
  # ----------------------------
  def _do_xform(self, obj, xform_procedure, **kwargs):
    # Get the transform method and invoke it
    xform_method = getattr(self, xform_procedure, None)
    assert(xform_method is not None)
    obj = xform_method(obj, **kwargs)
    return (obj)

  # ----------------------------
  # Dynamically apply any defined transforms on an item and return the resulting object
  # ----------------------------
  def _xform(self, obj):
    # Apply transforms on that object. The output of each transform becomes the input
    # for the next transform
    if (self.xforms is not None):
      for xform in self.xforms:
        obj = self._do_xform(obj, **xform)

    return obj

  # ----------------------------
  # The return is either a paired tuple or a list of paired tuples
  # [(obj, paired_obj), (obj, paired_obj), ...]
  # ----------------------------
  def get_paired_items(self, paired_objs, idxs):
    items = super().__getitem__(idxs)

    if isinstance(items,list):
      assert (len(items) == len(paired_objs))

      # Return a list of objects after applying transforms on each one
      objs = [(self._get_object(i), po) for i, po in zip(items, paired_objs)]
      return [self._xform(o) for o in objs]
    else:
      # Return a single object after applying transforms on it
      obj = (self._get_object(items), paired_objs)
      return self._xform(obj)

  # ----------------------------
  # Built-in dunder method to get an object or a list of objects from the ItemList, using
  # standard indexing and slicing. So 'idx' can be either a single index, a range or a slice
  # We get the data object, apply transforms on it and return the result
  # The return is an object or a list of objects
  # [obj, obj, ...]
  # ----------------------------
  def __getitem__(self, idxs):
    # Get the item (or list of items) corresponding to the given indices
    items = super().__getitem__(idxs)
    
    if isinstance(items,list):
      # Return a list of objects after applying transforms on each one
      objs = [self._get_object(i) for i in items]
      return [self._xform(o) for o in objs]
    else:
      # Return a single object after applying transforms on it
      obj = self._get_object(items)
      return self._xform(obj)

  # ----------------------------
  # While doing a split, we want to keep the original raw item without any transforms 
  # rather than the data object. So we skip the above __getitem__ method (which
  # gets the data object and transforms it)
  # !!!!!!!!!!!! There is no need for this special function to bypass the transforms
  # Instead of doing self[idx], you can do self.items[idx] which has the same effect
  # ----------------------------
  def _get_item_for_split_wo_xform(self, idx):
    # Get the raw item using the parent ListContainer class and return it without
    # applying any transforms
    raw = super().__getitem__(idx)
    return (raw)

  def get_display_data(self, obj, idx):
    return obj

### File ItemLists

In [ ]:
#export

#----------------------------------------------------
# An ItemList of File Paths, which are relative to 'folder_path'
# It can have subclasses for specific file types such as Image Files, Audio Files, Text Files etc
# It implements Split and Extract methods that are relevant to files
#----------------------------------------------------
class FileNameItemList(ItemList):
  def __init__(self, items, folder_path, **kwargs):
    file_list = [folder_path / Path(item) for item in items]
    # File paths in 'file_list' are relative to 'folder_path'
    self.folder_path = folder_path
    super().__init__(file_list)
    self._copy_new.append('folder_path')

  # ----------------------------
  # Split files into training, validation and test based on the sub-folder in which they reside
  # The 'train_folder' (and 'valid_folder' and 'test_folder') are relative to the base
  # 'folder_path' of the ItemList
  # ----------------------------
  def split_path(self, train_folder, valid_folder, test_folder=None):
    train_file_list, valid_file_list, test_file_list = [], [], []
    sub_folders = [train_folder, valid_folder, test_folder]
    file_lists = [train_file_list, valid_file_list, test_file_list]

    # Check each file path item in our list to see which sub folder it resides and
    # add it to the appropriate file list for training, validation and test
    for file_path in self._get_item_for_split_wo_xform(slice(None, None)):
      self.in_sub_folder(file_path, self.folder_path, zip(sub_folders, file_lists))

    # Use the list of items for training, validation and test to create three ItemLists 
    # Necessary attributes are copied over from the source ItemList
    train_rows = self.clone(train_file_list)
    valid_rows = self.clone(valid_file_list)
    test_rows = self.clone(test_file_list)

    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Get the n-th level ancestor folder of the given 'file_path' 
  #
  # !!!!!!!!! Add a built-in extract procedure 'extract_ancestor' which only works for FileName item lists
  # rather than use a custom function
  # ----------------------------
  @staticmethod
  def get_ancestor(file_path, up_level=1):
    sub_folder_path = file_path.parents[up_level - 1].name
    return (sub_folder_path)

  # ----------------------------
  # Check which sub-folder the 'file_path' resides in and add it to the corresponding list
  # ----------------------------
  @staticmethod
  def in_sub_folder(file_path, folder_path, zip_folder_files):
    # 'zip_folder_files' is [(sub_folder_tr, file_list_tr), (sub_folder_val, file_list_val), ...]
    # For each sub-folder, check if 'file_path' resides in 'folder_path' / 'sub_folder' and add
    # it to the corresponding file_list
    for sub_folder, file_list in zip_folder_files:
      if (sub_folder is not None):
        sub_folder_path = folder_path / sub_folder
        if (sub_folder_path in file_path.parents):
          file_list.append(file_path)

#----------------------------------------------------
# Subclass of File Name Item List for Image Files
# Doesn't do much by itself, most of the file-specific functionality is in the parent
# class and the image-specific functionality is contained in ImageItemList
#----------------------------------------------------
class ImageFileItemList(FileNameItemList):
  def FileToImage(self, items, convert_state, in_train, **kwargs):
    imagefile_items = items
    image_items = imagefile_items
    return (image_items)

#----------------------------------------------------
# Subclass of File Name Item List for Text Files
#----------------------------------------------------
class TextFileItemList(FileNameItemList):
  # Read the entire text file as one document into a single sentence
  @staticmethod
  def _read_doc(textfile_name):
    with open(textfile_name, 'r', encoding = 'utf8') as f: return f.read()

  # Read the text file line by line into a list of sentences
  @staticmethod
  def _read_sentence(textfile_name):
    print ('To be implemented - read the text file line by line')
    return(0)

  # ----------------------------
  # The entire doc in the text file becomes a single sentence item
  # ----------------------------
  def extract_doc(self, **kwargs):
    y_list = [self.__class__._read_doc(t) for t in self[:]]
    return (y_list)

  # ----------------------------
  # Each line in the text file becomes a separate sentence item
  # ----------------------------
  def extract_sentence(self):
    y_list = [self.__class__._read_sentence(t) for t in self[:]]
    return (y_list)

  # ----------------------------
  # Dummy extraction
  # ----------------------------
  def extract_dummy(self, **kwargs):
    y_list = [0] * len(self)
    return (y_list)

### Image ItemLists

In [ ]:
#export

#----------------------------------------------------
# ItemList for Images. Each item is an image file, which is dynamically loaded. It also
# implements several transforms which can then be applied in sequence to the loaded image.
#----------------------------------------------------
class ImageItemList(ItemList):
  def __init__(self, items, img_type='ocv', pair_type=None, **kwargs):
    self.img_type = img_type
    self.pair_type = pair_type
    self.mean = None
    self.std = None
    super().__init__(items, **kwargs)

  # ----------------------------
  # Load the file as an image
  # ----------------------------
  def _get_object(self, item):
    if (self.img_type == 'ocv'):
      return OcvImg.ocv_open(item)
      #return PIL.Image.open(item)

  # ----------------------------
  # Denormalise the tensor for display
  # ----------------------------
  def get_display_data(self, obj, idx):
    obj = self.denormalise(obj)
    return obj

  # ----------------------------
  # Decorator used with all transforms to handle paired input objects and paired
  # return values
  #
  # Since Image transforms support paired objects, the generic 
  # Item List's transform processing expects every Image transform to work in 
  # two modes:
  #   1. It gets passed a single image object 'img' and returns a single 
  #      transformed image 'img'
  #   2. It gets passed an (image, paired_image) tuple in the 'img' argument and 
  #      returns a transformed (image, paired_image) 'img' tuple
  #
  # In other words, the same 'img' argument can be either a single image or a tuple.
  #
  # Rather than requiring every transform to include the logic for the handling
  # the two modes, this logic is provided by this decorator. This simplifies every
  # transform so they can all work in one mode only:
  #   a) They always receive separate 'img' and 'img_pair' arguments. The 'img_pair' can
  #   be None.
  #   b) They always return separate transformed 'img' and 'img_pair' values. The
  #   returned 'img_pair' is None if the input 'img_pair' was None
  #
  # The decorator does the mapping between what the two modes that the generic 
  # processing expects and the single mode that the transform function supports.
  # ----------------------------
  def paired_xform(xform):
    # Note that the method above cannot take a 'self' argument. However the 'self' 
    # is available within the method and is passed to the 'pair' wrapper function.

    def pair(self, obj, *args, **kwargs):
      if (isinstance(obj, tuple)):
        # If a tuple is passed in, separate it into 'img' and 'img_pair' arguments
        img, img_pair = obj
      else:
        # If a single image is passed in, set the 'img_pair' to None
        img, img_pair = obj, None

      # Call the transform function
      res = xform(self, img, img_pair, *args, **kwargs)

      # If a single image was passed in, then return a single image and ignore
      # the 'img_pair' in the return value. Note that below we are checking the
      # input 'obj' for tupleness, not the 'res' result value. 
      if (not isinstance(obj, tuple)):
        res = res[0]

      return res
    return pair

  # ----------------------------
  # Transform for RGB
  # ----------------------------
  @paired_xform
  def make_rgb(self, img, img_pair):
    if (self.img_type == 'ocv'):
      img = OcvImg.ocv_rgb(img)
      return img, img_pair

  # ----------------------------
  # Transform for image resize
  # ----------------------------
  @paired_xform
  def resize(self, img, img_pair, size):
    assert(isinstance(size, int))
    if (self.img_type == 'ocv'):
      img = OcvImg.ocv_resize(img, (size, size))
      if (img_pair is not None):
        img_pair = OcvImg.ocv_resize(img_pair, (size, size))
      return img, img_pair

  # ----------------------------
  # Randomly choose from a set of rotate and flip transforms. We also have the
  # probability of doing nothing
  # ----------------------------
  @paired_xform
  def flip_rotate(self, img, img_pair, types=None, prob=0.75):
    if random.random() > prob:
      # Do nothing with some probability
      return img, img_pair

    elif (self.img_type == 'ocv'):
      if (types is None):
        types = ['rotate_90', 'rotate_180', 'rotate_270', 'fliv_h', 'fliv_v', 'transpose', 'transverse']
      fr_type = random.choice(types)
      img = OcvImg.ocv_flip_rotate(img, fr_type)
      if (img_pair is not None):
        img_pair = OcvImg.ocv_flip_rotate(img_pair, fr_type)
      return img, img_pair

    elif (self.img_type == 'pil'):
      if (types is None):
        types = ['rotate_90', 'rotate_180', 'rotate_270', 'fliv_h', 'fliv_v', 'transpose', 'transverse']
      fr_type = random.choice(types)
      img = PilImg.pil_flip_rotate(img, fr_type)
      if (img_pair is not None):
        img_pair = PilImg.pil_flip_rotate(img_pair, fr_type)
      return img, img_pair

  # ----------------------------
  # Transform for Centre Crop
  # ----------------------------
  @paired_xform
  def centre_crop(self, img, img_pair, crop_size):
    assert(isinstance(crop_size, int))
    if (self.img_type == 'ocv'):
      img_sz = OcvImg.ocv_shape(img)

      # Get the crop coordinates. We will apply those same coordinates to both
      # the 'img' and the 'img_pair'
      crop_corners = imgaug_random_resized_crop(img_sz, 
                                  crop_area_range=(0.7694, 0.7694), 
                                  crop_aspect_range=None, 
                                  crop_pos='ctr')
      
      # Crop the image
      img = OcvImg.ocv_crop(img, (crop_size, crop_size), crop_corners)

      if (img_pair is not None):
        # Crop the paired image
        img_pair = OcvImg.ocv_crop(img_pair, (crop_size, crop_size), crop_corners)

      return img, img_pair

  # ----------------------------
  # Transform for Random Resized Crop
  # ----------------------------
  @paired_xform
  def random_resized_crop(self, img, img_pair, crop_size):
    assert(isinstance(crop_size, int))
    if (self.img_type == 'ocv'):
      img_sz = OcvImg.ocv_shape(img)

      # Get the crop coordinates. We will apply those same coordinates to both
      # the 'img' and the 'img_pair'
      crop_corners = imgaug_random_resized_crop(img_sz)

      # Crop the image
      img = OcvImg.ocv_crop(img, (crop_size, crop_size), crop_corners)

      if (img_pair is not None):
        # Crop the paired image
        img_pair = OcvImg.ocv_crop(img_pair, (crop_size, crop_size), crop_corners)

      return img, img_pair

  # ----------------------------
  # Transform for Perspective Warp
  # ----------------------------
  @paired_xform
  def perspective_warp(self, img, img_pair, crop_size):
    assert(isinstance(crop_size, int))
    if (self.img_type == 'ocv'):
      img_sz = OcvImg.ocv_shape(img)

      # Get the warped coordinates. We will apply those same coordinates to both the
      # 'img' and the 'img_pair'. Note that with 'ocv', we use the src_coords and targ_coords
      # and ignore the 'coeffs' as they are not in a suitable format for OCV. We let the 
      # 'ocv_perspective' function calculate the matrix coefficients for the warp in the 
      # needed format for OCV, by using the src and targ coords.
      # On the other hand, With PIL, we use the matrix 'coeffs' directly as they are 
      # already calculated in the needed format for PIL.
      coeffs, src_coords, targ_coords = imgaug_perspective_warp(img_sz, (crop_size, crop_size), magnitude=0.2)

      # Warp the image
      img = OcvImg.ocv_perspective(img, (crop_size, crop_size), src_coords, targ_coords)

      if (img_pair is not None):
        # Warp the paired image
        img_pair = OcvImg.ocv_perspective(img_pair, (crop_size, crop_size), src_coords, targ_coords)

      return img, img_pair

  # ----------------------------
  # Tranform using a variety of augmentations provided by the Albumentations
  # library. The 'aug_name' chooses the augmentation we want to use.
  # ----------------------------
  @paired_xform
  def aug(self, img, img_pair, aug_name, p=0.5):
    if (self.img_type == 'ocv'):
      img, img_pair = imgaug_albu(img, aug_name, mask=img_pair, p=p)
      return img, img_pair

  # ----------------------------
  # Transform for converting image to a tensor of bytes
  # ----------------------------
  @paired_xform
  def to_byte_tensor(self, img, img_pair):
    if (self.img_type == 'ocv'):
      img = OcvImg.ocv_byte_tensor(img)
      return img, img_pair

  # ----------------------------
  # Transform for converting from a byte tensor to float tensor
  # ----------------------------
  @paired_xform
  def to_float_tensor(self, img_tensor, img_pair):
    img_tensor = img_tensor.float().div_(255.)
    if (img_pair is not None):
      img_pair = tensor(img_pair).div_(255.)
    return img_tensor, img_pair

  # ----------------------------
  # Set the mean and std to be used when normalising the image tensor. This is
  # called during data post processing, and is not a transform.
  #
  # mean and std are both lists, with one value per image channel depth.
  # If we're given pre-calculated 'mean' and 'std' then use them directly
  # If not, then calculate mean and std using the given data batch
  # ----------------------------
  def set_mean_std(self, datab, in_train, mean=None, std=None, **kwargs):

    # We can assume that if mean is None, std will also be None
    if ((mean is None) and in_train):
        # If this is a training dataset, calculate the mean and std from the 
        # given batch and save them in the 'kwargs' so that they get passed in the 
        # 'mean' and 'std' arguments to the validation dataset.
        mean = datab.mean()
        std = datab.std()
        kwargs['mean'] = mean
        kwargs['std'] = std
    
    # Save them for use when doing normalisation transforms
    self.mean, self.std = tensor(mean), tensor(std)

  # ----------------------------
  # Transform to normalise the image tensor, based on mean and std that were
  # pre-calculated earlier during data post processing
  # ----------------------------
  @paired_xform
  def normalise(self, img_tensor, img_pair):
    mean, std = self.mean, self.std

    # If normalise() is called, mean and std should have been pre-assigned
    # and should never be null. But there is one instance when it will be
    # null ie. when we fetch a batch from the DataLoader for the very first
    # time during data post-processing, in order to calculate the mean and std
    # This is a chicken-and-egg situation. So rather than figuring out a way
    # to set some flag to differentiate that case, we simply do nothing if
    # mean and std is not set. And we assume that, except for that first 
    # time, the caller has correctly set mean and std.
    if (mean is None):
      return img_tensor, img_pair

    assert(isinstance(img_tensor, torch.Tensor))
    assert(isinstance(mean, torch.Tensor) and (mean.size(0) == img_tensor.size(0)))
    assert(isinstance(std, torch.Tensor) and (std.size(0) == img_tensor.size(0)))

    norm = (img_tensor - mean[...,None,None]) / std[...,None,None]
    return norm, img_pair

  # ----------------------------
  # Denormalise the image tensor for display. This is not a transform
  # ----------------------------
  def denormalise(self, obj_tensor):
    mean, std = self.mean, self.std

    if (mean is None):
      # Do nothing if the tensor was not normalised
      return obj_tensor

    assert(isinstance(obj_tensor, torch.Tensor))
    assert(isinstance(mean, torch.Tensor) and (mean.size(0) == obj_tensor.size(0)))
    assert(isinstance(std, torch.Tensor) and (std.size(0) == obj_tensor.size(0)))

    denorm = (obj_tensor * std[...,None,None]) + mean[...,None,None]
    return denorm
    

### Text ItemLists

In [ ]:
#export

#----------------------------------------------------
# ItemList where each item is a text sentence
# eg. ["sentence 1", "sentence 2", ..]
#----------------------------------------------------
#special tokens
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()
def_pre_rules = ["fixup_text", "replace_rep", "replace_wrep", "spec_add_spaces", "rm_useless_spaces", "sub_br"]
def_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]
def_post_rules = ["deal_caps", "replace_all_caps", "add_eos_bos"]

class SentenceItemList(ItemList):
  def __init__(self, items, lang="en", pre_rules=def_pre_rules, spec_tok=def_spec_tok, post_rules=def_post_rules, **kwargs):
    self.pre_rules, self.spec_tok, self.post_rules = pre_rules, spec_tok, post_rules
    self.chunksize, self.max_workers = 2000, 4
    self.tokenizer = spacy.blank(lang).tokenizer
    for w in self.spec_tok:
      self.tokenizer.add_special_case(w, [{ORTH: w}])
    super().__init__(items)
  
  @staticmethod
  def sub_br(t):
    "Replace <br > or <br /> by \n"
    # Pattern is '<', then 0 or more spaces, then 'br', then 0 or more spaces, then 0 or 1 '/', then '>'
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

  @staticmethod
  def spec_add_spaces(t):
    "Add spaces around / and #"
    return re.sub(r'([#/])', r' \1 ', t)
  
  @staticmethod
  def rm_useless_spaces(t):
    "Remove multiple spaces"

    # Look for 1 or more occurrences of a space and replace with a single space. This works
    # but results in a replacement even for single spaces, which are quite common. So the
    # alternate expression below is more efficient
    #return re.sub(r'[\s]+', r' ', t)

    # Look for 2 or more occurrences of a space ie. ' ' and replaces with a single space
    return re.sub(' {2,}', ' ', t)  

  @staticmethod
  def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"

    # Use a replacement function to find the number of repeating characters
    def replace_func(m):
      c = m.group()

      # len(c) - number of repeats, c[0] - repeating character
      return (f' {TK_REP} {len(c)} {c[0]} ')
  
    # Find any non-space characters which repeat 4 or more times. 
    # The (\S) finds the first character, and then the \1{3,} looks for 3 more repeats ie. total of 4 repeats = 1 + 3
    # NB: Alternately '(\S)\1+' - will find repeats of 2 or more times ie. 1 + 1
    re_rep = re.compile(r'(\S)\1{3,}')
    return re_rep.sub(replace_func, t)

  @staticmethod
  def replace_wrep(t):
    "Replace word repetitions: word word word -> TK_WREP 3 word"

    # Use a replacement function to find the number of repeating words
    def replace_func(m):
      w = m.group()
    
      # Make a list of repeating words
      ws = w.split()

      # len(ws) - number of repeats, ws[0] - repeating word
      return (f' {TK_WREP} {len(ws)} {ws[0]} ')
  
    # Find any words which repeat 4 or more times. 
    # The (\b\w+\W+) finds an empty string marking a word boundary, followed by multiple word characters, followed by multiple non-word characters
    # then the \1{3,} looks for 3 more repeats ie. total of 4 repeats = 1 + 3
    re_rep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_rep.sub(replace_func, t)

  @staticmethod
  def fixup_text(x):
    "Various messy things we've seen in documents"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

  @staticmethod
  def replace_all_caps(x):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

  @staticmethod
  def deal_caps(x):
    "Replace all Capitalized tokens in by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

  @staticmethod
  def add_eos_bos(x): return [BOS] + x + [EOS]

  #----------------------------------------------------
  # Apply all the tokenising rules
  # Pre-rules are applied before a sentence is tokenised into words
  # Post-rules are applied after
  #----------------------------------------------------
  def apply_rules(self, sentence, pre):
    rules = self.pre_rules if pre else self.post_rules
    for rule in rules:
      rule_method = getattr (self.__class__, rule, None)
      assert(rule_method is not None)
      sentence = rule_method(sentence)

    return (sentence)

  #----------------------------------------------------
  # Break a sentence into word tokens
  #----------------------------------------------------
  def tokenise_sentence(self, args):
    i, sentence_items = args
    sentence_items = [self.apply_rules(sentence, pre=True) for sentence in sentence_items]
    sentence_word_items = [[d.text for d in doc] for doc in self.tokenizer.pipe(sentence_items)]
    sentence_word_items = [self.apply_rules(sentence, pre=False) for sentence in sentence_word_items]
    return (sentence_word_items)

  #----------------------------------------------------
  # 
  #----------------------------------------------------
  def SentenceToWord(self, items, convert_state, in_train, **kwargs):
    toks = []
    sentence_items = items
    chunks = [sentence_items[i: i+self.chunksize] for i in (range(0, len(sentence_items), self.chunksize))]
    toks = parallel(self.tokenise_sentence, chunks, max_workers=self.max_workers)
    return sum(toks, [])

#----------------------------------------------------
# ItemList where each item is a list of text words corresponding to one sentence
# eg. [["word1", "word2", "word3"], ["word4", "word5", "word6", "word7"], ...]
#----------------------------------------------------
class SentenceWordItemList(ItemList):
  def __init__(self, items, **kwargs):
    self.max_vocab, self.min_freq = 60000, 2
    self.spec_tok = def_spec_tok
    super().__init__(items)

  #----------------------------------------------------
  # Numericalise the word tokens into word IDs using a vocab of words to IDs
  # We build the vocab during training (ie. in_train is True) unless a pre-created vocab
  # is passed in. We save the vocab in the 'convert_state', so we can refer to it later.
  # During validation, we use the saved vocab that was built during training.
  #----------------------------------------------------
  def WordToWordId(self, items, convert_state, in_train, vocab_i2w=None, **kwargs):
    sentence_word_items = items
    if (in_train):
      if (vocab_i2w is None):
        # Build a vocab of unique words and save it for use during validation processing
        freq = Counter(word for sentence in sentence_word_items for word in sentence)
        vocab_i2w = [word for word,count in freq.most_common(self.max_vocab) if count >= self.min_freq]
        for tok in reversed(self.spec_tok):
          if tok in vocab_i2w: vocab_i2w.remove(tok)
          vocab_i2w.insert(0, tok)
      convert_state['vocab_i2w'] = vocab_i2w

      # Build a mapping between word and IDs and save it. By using a defaultdict, any
      # words which are not in the vocab don't get a Key Error but get an ID value of 0 which is UNK
      vocab_w2i = collections.defaultdict(int, {word:id for id, word in enumerate(vocab_i2w)})
      convert_state['vocab_w2i'] = vocab_w2i
    else:
      # Get the mapping built during training processing
      vocab_w2i = convert_state['vocab_w2i']

    # Use the mapping to get an ID for each word item
    # Because vocab_w2i is a defaultdict, when the 'map' looks up any words which are not in the 
    # vocab, they will get added into vocab_w2i
    wordid_items = [list(map(lambda x: vocab_w2i[x], sentence)) for sentence in sentence_word_items]

    return (wordid_items)

#----------------------------------------------------
# ItemList where each item is a list of word IDs corresponding to one sentence
# eg. [[27, 9, 47], [16, 3, 14, 86], ...]
#----------------------------------------------------
class SentenceWordIdItemList(ItemList):
  def __init__(self, items, vocab_i2w, **kwargs):
    super().__init__(items)
    self.vocab_i2w = vocab_i2w

  def WordIdToStream(self, items, convert_state, in_train, **kwargs):
    sentence_wordid_items = items
    stream_items = [wordid for sentence in sentence_wordid_items for wordid in sentence]
    return (stream_items)

  def get_display_data(self, obj, idx):
    sentence = obj
    words = [self.vocab_i2w[wordid] for wordid in sentence]
    sentence = ' '.join(words)
    return sentence

#----------------------------------------------------
# ItemList which is a contiguous stream of word IDs
# eg. [27, 9, 47, 16, 3, 14, 86, ...]
#----------------------------------------------------
class StreamWordIdItemList(ItemList):
  def __init__(self, items, vocab_i2w, **kwargs):
    super().__init__(items)
    self.vocab_i2w = vocab_i2w

  def get_display_data(self, obj, idx):
    # Normally, with a single 'idx' for a Stream item list we would be given a single word id 'obj'
    # so we convert it to a list to be able to iterate over it.
    #
    # On the other hand, if a range of slice 'idx' values was used, we would be given a
    # list 'obj' already ready for iteration, so no conversion is necessary here. Also, in the 
    # case when the Stream item list is contained within a FastaiLMDataset, the 'obj' 
    # is a tensor of a chunk of text, which is also ready for iteration. 
    text = obj
    if (isinstance(text, int)): text = list(text)

    words = [self.vocab_i2w[wordid] for wordid in text]
    sentence = ' '.join(words)
    return sentence

### Audio Item List

In [ ]:
!pip install torchaudio

In [ ]:
#export

from nb_audio import AudioUtil

#----------------------------------------------------
# Subclass of File Name Item List for Audio Files
# Doesn't do much by itself, most of the file-specific functionality is in the parent
# class and the audio-specific functionality is contained in AudioItemList
#----------------------------------------------------
class AudioFileItemList(FileNameItemList):
  def FileToAudio(self, items, convert_state, in_train, **kwargs):
    audiofile_items = items
    return (audiofile_items)

#----------------------------------------------------
# ItemList for Audio. Each item is an audio file, which is dynamically loaded. It also
# implements several transforms which can then be applied in sequence to the loaded audio data.
#----------------------------------------------------
class AudioItemList(ItemList):
  def __init__(self, items, **kwargs):
    super().__init__(items, **kwargs)

  # ----------------------------
  # Load the file as audio data
  # ----------------------------
  def _get_object(self, item):
    return AudioUtil.open(item)

  # ----------------------------
  # 
  # ----------------------------
  def get_display_data(self, obj, idx):
    return obj

  # ----------------------------
  # Transform to pad or trim the audio to a fixed length (in milliseconds), since 
  # the model needs all tensors to be the same size
  # ----------------------------
  def pad_trim(self, aud, max_ms):
    return AudioUtil.pad_trim(aud, max_ms)

  # ----------------------------
  # Transform to shift the audio signal. Can be used for Augmentation of the
  # raw audio.
  # ----------------------------
  def signal_shift(self, aud, max_shift_pct=.6):
    return AudioUtil.signal_shift(aud, max_shift_pct)

  # ----------------------------
  # Transform to generate a Spectrogram
  # ----------------------------
  def spectro_gram(self, aud, spectro_type='mel', n_mels=64, n_fft=1024, hop_len=None):
    return AudioUtil.spectro_gram(aud, spectro_type, n_mels, n_fft, hop_len)

  # ----------------------------
  # Transform to randomly mask some frequencies or time steps of the Spectrogram.
  # Can be used for Augmentation of the processed Spectrogram rather than the raw
  # audio.
  # ----------------------------
  def spectro_augment(self, spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    return AudioUtil.spectro_augment(spec, max_mask_pct, n_freq_masks, n_time_masks)

### Other ItemLists

In [ ]:
#export

#----------------------------------------------------
# ItemList for label class names
# eg. ["class1", "class2", ...]
#----------------------------------------------------
class ClassNameItemList(ItemList):

  # ----------------------------
  # Convert class names to class IDs
  # During preparation of training data we build a list of unique class names and a mapping from names to IDs
  # During preparation of validation data we reuse the mapping built for training
  # ----------------------------
  def NameToId(self, items, convert_state, in_train, **kwargs):
    if (in_train):
      # Build a sorted list of unique class names and save it for use during validation processing
      class_names = list(set(items))
      class_names.sort()
      convert_state['class_vocab_i2n'] = class_names

      # Build a mapping between class name and IDs and save it
      class_vocab_n2i = {name:id for id, name in enumerate(class_names)}
      convert_state['class_vocab_n2i'] = class_vocab_n2i
    else:
      # Get the mapping built during training processing
      class_vocab_n2i = convert_state['class_vocab_n2i']

    # Use the mapping to get an ID for each class name item
    classid_items = list(map(lambda x: class_vocab_n2i[x], items))
    return (classid_items)

#----------------------------------------------------
# ItemList for label class IDs
# eg. [1, 2, ...]
#----------------------------------------------------
class ClassIdItemList(ItemList):
  def __init__(self, items, class_vocab_i2n, **kwargs):
    super().__init__(items)
    self.class_vocab_i2n = class_vocab_i2n

  def get_display_data(self, obj, idx):
    id = obj
    name = self.class_vocab_i2n[id]
    return name

#----------------------------------------------------
# ItemList for a Pandas Dataframe. Each row is an item. Since the actual data 
# is stored in the dataframe, the items themselves are just a range of numbers 
# from 0 to 'dataframe_length - 1'.
#----------------------------------------------------
class DfItemList(ItemList):
  # ----------------------------
  # Initialisation is called in several cases - after loading from file, after
  # splitting into training, validation and test sub-lists, after extracting into
  # 'x' and 'y' sub-lists and potentially after converting from one type to another.
  # 
  # In these different cases, the 'true' source of data is either:
  #   a) The populated 'df' OR
  #   b) A list of items. That list is used to select a subset of the 'df'
  # We have to handle these different cases and create a 'df' and list of 'items' 
  # that are consistent with each other.
  # ----------------------------
  def __init__(self, items, df, **kwargs):
    # If present, 'items' should never be longer than the Dataframe
    len_df = len(df.index)
    assert (not (items and (len(items) > len_df)))

    if (items and (len(items) < len_df)):
      # Items contains the indexes of a subset of the Dataframe. This happens when we
      # split the ItemList. Create a new Dataframe with those indexes
      df = df.iloc[items, :].reset_index(drop=True)
      len_df = len(df.index)

    self.df = df
    # Items should be a range of consecutive numbers from 0 to len_df - 1 
    items = list(range(len_df))
    super().__init__(items)

    df_mem = df.memory_usage().sum() / 1024**2
    items_mem = sys.getsizeof(items) / 1024**2
    print (f'DfItemList init with {len_df} items, {df_mem:.2f} df memory, {items_mem:.2f} items memory')

    self._copy_new.append('df')

  # ----------------------------
  # Read a CSV file into a Dataframe
  # ----------------------------
  @classmethod
  def open_csv(cls, csv_path):
    df = pd.read_csv(csv_path, low_memory=False)
    return df

  # ----------------------------
  # To get the actual data object, we use the item as an index into the
  # dataframe's rows. Therefore item #n corresponds to dataframe row #n.
  # ----------------------------
  def _get_object(self, i):
    return self.df.iloc[i]

  # ----------------------------
  # Extract data as a Python list using a column name
  # It is used to extract data out of a DataFrame into another target class
  # !!!!!!!!! Should be a list of columns
  # ----------------------------
  def extract_colval(self, col, **kwargs):
    y_list = self.df[col].tolist()
    return (y_list)

  # ----------------------------
  # Select a list of column names, or all columns except a list of column names
  # It is used to select data but the target type remains a DataFrame
  # ----------------------------
  def extract_select(self, cols=None, cols_exclude=None, extra_args={}, **kwargs):
    assert(not((cols is None) and (cols_exclude is None)))

    if (cols_exclude):
      cols = list(set(self.df.columns) - set(cols_exclude))

    extra_args['df'] = self.df[cols].copy()

    items = None
    return (items)

#----------------------------------------------------
# 
#----------------------------------------------------
class DummyItemList(ItemList):
  def __init__(self, items, **kwargs):
    super().__init__(items)

### Tabular Data

#### Tabular ItemList

In [ ]:
#export

import numpy as np

#----------------------------------------------------
# ItemList for Tabular data held in a Pandas Dataframe. Each row is an item.
# Optionally, we could be given additional Dataframes with data whose rows have 
# a 1:1 or a 1:many relationship with the main Dataframe.
#----------------------------------------------------
class TabularItemList(DfItemList):
  def __init__(self, items, df, related_dfs=None, fd={}, **kwargs):
    super().__init__(items, df, **kwargs)
    self.related_dfs = related_dfs

    self.fd = fd
    self._copy_new.append('fd')

    if (related_dfs is not None):
      self._save_related(df, related_dfs)
      self._copy_new.append('related_dfs')

  # ----------------------------
  # Read a CSV file into a Dataframe
  # ----------------------------
  @classmethod
  def open_csv(cls, csv_path):
    df = super().open_csv(csv_path)
    df = cls.reduce_mem(df, csv_path)
    return df

  # ----------------------------
  # Select the type of columns ( either categorical + continuous, or target 
  # columns) to be extracted.
  # It is used to select data but the target type remains a DataFrame
  # ----------------------------
  def extract_type(self, cols_type, extra_args={}, **kwargs):
    assert(cols_type in ['cat_cont', 'tgt'])

    cols = None
    if (cols_type == 'cat_cont'):
      cat_cols, cont_cols = self._categorical_cols(), self._continuous_cols()
      cols = cat_cols + cont_cols
      fd = {'nominal': cat_cols, 'continuous': cont_cols}
    elif (cols_type == 'tgt'):
      cols = self._target_cols()
      fd = {'target' : cols}

    extra_args['df'] = self.df[cols].copy()
    extra_args['fd'] = fd

    items = None
    return (items)

  # ----------------------------
  # To get the actual data object, we use the item as an index into the
  # numpy array's rows. Therefore item #i corresponds to array row #i.
  # ----------------------------
  def _get_object(self, i):
    return self.arr[i]

  #----------------------------------------------------
  # Define which columns are:
  #   Categorical - Nominal or Ordinal
  #   Numeric - Continuous or Discrete
  #   Target - ie. the dependent feature
  #
  # The given 'features_dict' is appended to any existing features_dict
  #
  # features_dict = {
  #    'key'     : key_columns,
  #    'primary' : primary_features,
  #    'nominal' : nominal_features,
  #    'ordinal' : ordinal_features,
  #    'continuous' : numeric_continuous_features,
  #    'discrete' : numeric_discrete_features,
  #    'target' : target_feature}
  #----------------------------------------------------
  def add_features_dict(self, features_dict, df_name=None, mds=None):
    fd = self._get_features_dict(df_name)

    # If a metadata summary has been given, get the categorical and continuous
    # columns from it
    if (mds is not None):
      # Mark the key and target columns in the mds so they get excluded from the 
      # categorical and continuous lists

      if ('key' in features_dict):
        key_cols = features_dict['key']
        mds.loc[key_cols, 'col_type'] = 'key'
    
      if ('target' in features_dict):
        target_cols = features_dict['target']
        mds.loc[target_cols, 'col_type'] = 'target'

      if ('continuous' not in features_dict):
        features_dict['nominal'] = mds[mds['col_type'] == 'categorical'].index.to_list()

      if ('continuous' not in features_dict):
        features_dict['continuous'] = mds[mds['col_type'] == 'continuous'].index.to_list()

    # Now update our features dict
    fd.update(features_dict)

  def _get_features_dict(self, df_name=None):
    if (df_name is None):
      return self.fd
    else:
      return getattr(self, f'{df_name}_fd', None)

  def _get_features_dict_cols(self, col_type, df_name=None):
    fd = self._get_features_dict(df_name)
    if (col_type in fd.keys()):
      return fd[col_type]
    else:
      return []

  def _key_cols(self):
    return self._get_features_dict_cols('key')

  def _primary_cols(self):
    return self._get_features_dict_cols('primary')

  def _numeric_cols(self):
    return self._get_features_dict_cols('continuous') + self._get_features_dict_cols('discrete')

  def _continuous_cols(self):
    return self._get_features_dict_cols('continuous')

  def _discrete_cols(self):
    return self._get_features_dict_cols('discrete')

  def _categorical_cols(self):
    return self._get_features_dict_cols('nominal') + self._get_features_dict_cols('ordinal')

  def _target_cols(self):
    return self._get_features_dict_cols('target')

  # ----------------------------
  # Save the ItemList for test data for later use during splitting
  # ----------------------------
  def add_test_rows(self, test_rows):
    self.test_rows = test_rows

  # ----------------------------
  # Split the rows into train and validation data sequentially, while the test data
  # that we saved previously can be returned unchanged
  # ----------------------------
  def split_sequential(self, test_data=False, **kwargs):
    train_rows, valid_rows, test_rows = super().split_sequential(**kwargs)
    if ('test_ratio' not in kwargs):
      test_rows = self._split_test(test_data)
    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Split the rows into train and validation data based on row indexes, while the test data
  # that we saved previously can be returned unchanged
  # ----------------------------
  def split_idxs(self, test_data=False, **kwargs):
    train_rows, valid_rows, _ = super().split_idxs(**kwargs)
    if ('test_ratio' not in kwargs):
      test_rows = self._split_test(test_data)
    return (train_rows, valid_rows, test_rows)

  # ----------------------------
  # Return the test data that was loaded previously
  # ----------------------------
  def _split_test(self, test_data=False):
    if (test_data):
      test_rows = getattr(self, 'test_rows', None)
    else:
      test_rows = None
    return test_rows

  # ----------------------------
  # Save each related df in its own attribute on the ItemList
  # ----------------------------
  def _save_related(self, df, related_dfs):
    # Save each related df in its named field
    for name, rdf in related_dfs.items():
      setattr(self, f'{name}_df', rdf)
      setattr(self, f'{name}_fd', {})

  # ----------------------------
  # Get the main df or a related df
  # ----------------------------
  def _get_df(self, df_name=None):
    if (df_name is None):
      # Use main df
      df = self.df
    else:
      # Use related df
      df = getattr(self, f'{df_name}_df', None)
    
    return df

  # ----------------------------
  # Set the main df or a related df
  # ----------------------------
  def _set_df(self, res_df, df_name=None):
    if (res_df is None): return

    if (df_name is None):
      # Update the main df
      self.df = res_df
    else:
      # Update the related df
      setattr(self, f'{df_name}_df', res_df)

  # ----------------------------
  # Flatten the columns of a multi-level column index, that gets created as a
  # result of a crosstab, pivot_table or groupby aggregation.
  # ----------------------------
  @staticmethod
  def _flatten_col_names(df):
    if (df.columns.nlevels > 1):
      df.columns = ['_'.join(col) for col in df.columns.values]

  # ----------------------------
  # Decorator for all data processor functions.
  #
  # All the processor functions are kept simple and take an explicit 'df' argument 
  # and may return a 'df' as well. They don't contain logic to figure out which df
  # to use from amongst the ItemList's main df and all the related dfs. Similarly
  # if they return a result df, they don't contain logic to figure out which of the
  # ItemList's dfs should be updated.
  #
  # That logic is encapsulated within this decorator. It has three modes. The first
  # two modes are the only ones that can be used when it is invoked externally. The
  # third mode is only used internally when one processor invokes another processor 
  # method within the class:
  #   1. The simple case, where no 'df' is passed in. Decorator uses the ItemList's main df. 
  #   2. A 'df_name' is passed in. Decorator uses the corresponding related df.
  #   3. A 'df' is passed in when invoked internally. The Decorator then bypasses all
  #      its logic, and just invokes the function and returns its result normally.
  # 
  # It also takes some flags, so that the behaviour can be different for the converter
  # data processors and for the EDA display utilities.
  #   1. save_res - if True, the result df is saved and replaces the df used above
  #   2. use_fd - if True, the feature dict for the df is also passed in the kwargs
  # ----------------------------
  def df_process(save_res=True, use_fd=False):
    def name_to_df(proc):
      # Note that the method above cannot take a 'self' argument. However the 'self' 
      # is available within the method and is passed to the 'pair' wrapper function.

      def wrap(self, df=None, df_name=None, *args, **kwargs):
        if (df is not None):
          # Bypass all decorator logic
          return proc(self, df, *args, **kwargs)
        
        # items, convert_state, in_train, df_name

        extra_dfs = {}

        # Get the df to process
        df = self._get_df(df_name)
        # Pass the feature dictionary as well
        if (use_fd):
          fd = self._get_features_dict(df_name)
          extra_dfs['fd'] = fd

        # Check all other keyword arguments for anything named 'xxx_df_name' and
        # if so, find the corresponding related df, and add an argument with the 
        # prefix {'xxx': <related_df>}
        for kwarg, kwargv in kwargs.items():
          if (kwarg.endswith('df_name')):
            kwarg_pre = kwarg.partition('df_name')[0]
            extra_dfs[f'{kwarg_pre}df'] = self._get_df(kwargv)
            if (use_fd):
              extra_dfs[f'{kwarg_pre}fd'] = self._get_features_dict(kwargv)

        # We added extra arguments to be passed in
        if (extra_dfs): kwargs.update(extra_dfs)

        # Call the processor function
        res_df = proc(self, df, *args, **kwargs)

        # Save the result df if it is returned
        if (save_res):
          self._set_df(res_df, df_name)
        else:
          return res_df

        # return items

      return wrap
    return name_to_df

  # PREPARE Utilities for pre-processing
  # ===============================================================

  # ----------------------------
  # Convert a numeric column that represents True/False with 1s and 0s into a
  # bool column
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def num_01_to_bool(self, df, col):
    df[col] = df[col] != 0
    return df

  # ----------------------------
  # Sort by columns
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def sort_by(self, df, sort_cols, ascending):
    df = df.sort_values(sort_cols, ascending=ascending)
    return df

  # ----------------------------
  # Remove a column. If 'col_ys' is True, remove any columns with names ending
  # with '_y', as they were duplicates after merging
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def remove_col(self, df, cols, col_ys=True):
    # Find columns ending with '_y'
    if (col_ys):
      for col in df.columns:
        if col.endswith('_y'):
          cols.append(col)

    # Now remove all the required columns
    df.drop(columns=cols, inplace = True)

  # ----------------------------
  # Rename a column
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def rename_col(self, df, col, new_name):
    df = df.rename(columns={col: new_name})
    return df

  # ----------------------------
  # Remap old category values to new values
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def remap_val(self, df, col, remap_dict):
    df[col] = df[col].map(remap_dict)

  # ----------------------------
  # Clone the index (containing row IDs) and add it as a new column
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def add_index_col(self, df, idx_col):
    df.index = df.index.set_names([idx_col])
    df = df.reset_index()
    return df

  # ----------------------------
  # Do a Left Outer Join on the given 'left_on' and 'right_on' fields
  # 'suffixes' describes the naming convention for duplicate fields. 
  # We leave the duplicate field names on the left untouched, and append a "_y" to those on the right
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def join_df(self, left_df, right_df, left_on, right_on=None, how='left', suffix='_y', right_key=None, **kwargs):
    if right_on is None: right_on = left_on
    join_df = left_df.merge(right_df, how=how, left_on=left_on, right_on=right_on, 
                        suffixes=("", suffix))

    if (right_key is None): right_key = right_df.columns[0]
    assert(len(join_df[join_df[right_key].isnull()]) == 0)

    return join_df
    
  # ----------------------------
  # Aggregate columns. Group by 'group_cols' and then aggregate the columns in 'agg_cols_fn'
  # using the given functions
  # 
  # NB: rollup() provides the same functionality, and is the preferred way as 
  # it uses pivot_tables rather than groupby.
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def _rollup_groupby(self, df, group_cols, agg_cols_fn):
    # Expand the types of the aggregate columns so that the aggregates don't overflow
    # NB: !!!!!! we are changing the original dataframe. should we do it on a copy instead?
    agg_cols = list(agg_cols_fn.keys())
    df[agg_cols] = df[agg_cols].astype('float64')

    # Group by 'group_cols'
    gd = df.groupby(group_cols)

    # Do the aggregates
    adf = gd.agg(agg_cols_fn)

    # The columns are now a two-level index, with the first level being the original
    # column name and the second level being the aggregation function name.
    # Rename the columns by flattening the index to a single level and appending 
    # the column name to the aggregation function name.
    self._flatten_col_names(adf)

    adf = adf.reset_index()
    return adf
  
  # ----------------------------
  # Helper function to rollup categorical columns
  # NB: _roll_cat() is the preferred way as it is much faster and uses get_dummies 
  # rather than looping with crosstab
  # ----------------------------
  def _roll_cat_crosstab(self, df, index, cat_cols):
    concat_dfs = []
    # We want separate rolled up counts for each categorical column. Doing this in
    # a single pivot_table() (or crosstab()) call by passing the list of categorical
    # columns results in a deep multi-level column index with counts for all possible
    # combinations of those column values, which is not what we want. So we get counts
    # for one column at a time and concatenate them all into a single result df.
    for col in cat_cols:
      # Use crosstab since pivot_table doesn't support the normalise feature, 
      # which gives us percentage counts, by row
      indexes = [df[idx] for idx in index]
      df_count = pd.crosstab(indexes, df[col], normalize='index')
      df_count.columns = [f'{df_count.columns.name}_{val}' for val in df_count.columns.values]

      # Add to our list of count dfs
      concat_dfs.append(df_count)

    # Concatenate all our count dfs column-wise
    df_cat = pd.concat(concat_dfs, axis=1, verify_integrity=True)
    return df_cat

  # ----------------------------
  # Helper function to rollup categorical columns 
  # ----------------------------
  def _roll_cat(self, df, index, cat_cols, nan_as_category=False):
    # Create one column per categorical value, of each categorical column
    original_columns = list(df.columns)
    df = pd.get_dummies(df, columns=cat_cols, dummy_na=nan_as_category)

    # Just use .mean() instead of agg_cat
    # Use df_cat throughout not 'df'. Can get rid of prior columns. Then no
    # need for new_columns.
    # Also use _flatten_col_names

    new_columns = [c for c in df.columns if c not in original_columns]
    agg_cat = {}
    for cat in new_columns:
        agg_cat[cat] = ['mean']
    df_cat = df.groupby(index).agg(agg_cat)
    #df_cat.columns = ['_'.join(col) for col in df_cat.columns.values]
    df_cat.columns = new_columns

    return df_cat

  # ----------------------------
  # Helper function to rollup continuous columns 
  # ----------------------------
  def _roll_cont(self, df, index, group_cols, agg_cont):
    # Probably due to a Pandas bug, columns of type 'float16' result in a 
    # bizarre 'No Matching Signature' error. Converting to 'float32' gets rid
    # of the error
    for col in agg_cont.keys():
      if (df[col].dtype == 'float16'):
        df[col] = df[col].astype('float32')

    # Aggregate the continuous columns, optionally grouping by some categorical
    # columns. Then flatten the multi-index column names
    df_cont = df.pivot_table(index=index, columns=group_cols, aggfunc=agg_cont)
    self._flatten_col_names(df_cont)
    return df_cont

  # ----------------------------
  # Rollup a related df by aggregating the categorical and continuous columns 
  # so that it can be joined with the main df
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def rollup(self, df, index, group_cols, cat_cols, agg_cont):
    # Rollup categorical and continuous columns separately
    concat_dfs = []
    if (cat_cols):
      df_cat = self._roll_cat(df, index, cat_cols)
      concat_dfs.append(df_cat)
    if (agg_cont):
      df_cont = self._roll_cont(df, index, group_cols, agg_cont)
      concat_dfs.append(df_cont)

    # and then concatenate them column-wise
    rdf = pd.concat(concat_dfs, axis=1, verify_integrity=True)
    return rdf

  # ----------------------------
  # Convert a field to Datetime
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def to_date(self, df, col, **kwargs):
    fld_dtype = df[col].dtype
      
    # If it is a custom datatime datatype, change the type to the standard datetime
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
      print ('Found DatetimeTZDtype ', col)
      fld_dtype = np.datetime64

    # If the column is a Pandas categorical type, then Numpy's np.issubdtype() throws an exception.
    # Also, for categorical types, for some reason in some cases, to_datetime() doesn't convert 
    # to datetime correctly. So we convert categorical to object beforehand to prevent these errors.
    if (pd.api.types.is_categorical_dtype(fld_dtype)):
      df[col] = df[col].astype('object')

    # If it is not already a datetime datatype, convert it to datetime
    if not np.issubdtype(fld_dtype, np.datetime64):
      df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

  # ----------------------------
  # Extract separate feature columns for all relevant attributes of DateTime fields
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def extract_date(self, df, col, drop=True, time=False, attr=None, **kwargs):
    # First make sure the field is converted to Datetime
    self.to_date(df=df, col=col)

    fld = df[col]

    # Get any prefix in the field name before the word 'Date'
    # We will pre-pend this prefix to any new column names that we extract below
    targ_pre = re.sub('[Dd]ate$', '', col)
      
    # List of attributes for which we want to extract separate columns
    if (attr is None):
      attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
              'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    # Include attributes for time in the list
    if time: attr = attr + ['Hour', 'Minute', 'Second']
      
    # Create a separate column for each attribute.
    # Column name is created by concatentating the 'prefix' and the 'attribute'
    # All datetime columns have ".dt" from which we extract all the required attributes
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
      
    # Create an Elapsed column
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
      
    # Drop the original date column if needed
    if drop: df.drop(col, axis=1, inplace=True)

  # ----------------------------
  # Calculate days since some event
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def time_since(self, df, sort_col, date_col, col, prefix):
    secs_per_day = np.timedelta64(1, 'D')
    last_event_date = np.datetime64()
    last_sort_val = 0
    res = []

    for s,v,d in zip(df[sort_col].values, df[col].values, df[date_col].values):
      # Reset counters at next sort value
      if s != last_sort_val:
        last_event_date = np.datetime64()
        last_event_date = None
        last_sort_val = s

      # Event occurs
      if v: last_event_date = d

      # Number of days since last occurrence of event
      if (last_event_date is None):
        # Fill 0 until the first event occurs
        res.append(int(0))
      else:
        res.append(((d - last_event_date).astype('timedelta64[D]') / secs_per_day))

    df[prefix + col] = res

  # ----------------------------
  # Aggregate columns over rolling time windows.
  # Compute rolling time windows on 'date_col' of duration 'window_sz'. Optionally
  # also group by 'group_cols' if given. Then aggregate 'agg_cols'
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def rolling_window(self, df, date_col, group_cols, window_sz, agg_cols, agg_fn='sum', ascending=True, suffix=None):
    # Sort by the 'date_col'
    gd = df.set_index(date_col).sort_index(ascending=ascending)

    # Group by 'group_col's if given
    if (group_cols is not None):
      gd = gd.groupby(group_cols)
 
    # Compute rolling windows
    gr = gd.rolling(window_sz, min_periods=1)

    # Do the aggregates
    adf = gr[agg_cols].agg(agg_fn)
    adf = adf.reset_index()

    # 'adf' now contains only the aggregates but not the original 'agg_cols' columns.
    # So merge the aggregates into the original 'df'
    join_cols = group_cols if (group_cols is not None) else []
    join_cols.append(date_col)
    suffix = suffix if (suffix is not None) else '_' + agg_fn
    df = df.merge(adf, how='left', left_on=join_cols, right_on=join_cols, suffixes=('', suffix))

    return df

  # ----------------------------
  # Split a text value using a separator into separate text columns. 
  # The names of the new columns created is passed as a 'new_cols' list 
  #    eg. ['new_col_name_2', None, 'new_col_name_2', ...]. Any new column which 
  # we don't want can be passed as a None.
  # Optionally, the original text column can be dropped.
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def text_split(self, df, col, sep, new_cols, drop_col=True, **kwargs):
    # Split the string, and expand each part as a separate column
    ndf = df[col].str.split(sep, expand=True)

    # Assign names for each new column created, or exclude it if it is None
    for i, new_col in enumerate(new_cols):
      if (new_col is not None):
        df[new_col] = ndf[i]

    # Drop the original column if required
    if (drop_col):
      df.drop(columns=[col], inplace = True)

  # ----------------------------
  # Replace a text value by another value
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def text_replace(self, df, col, old_val, new_val, **kwargs):
    df.loc[df[col] == old_val, col] = new_val

  # ----------------------------
  # Concatenate multiple text values
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def text_concat(self, df, new_col, concat_cols, space=' ', **kwargs):
    assert(isinstance(concat_cols, list) and (len(concat_cols) >= 2))
    df[new_col] = df[concat_cols[0]] + space + df[concat_cols[1]]
    for concat_col in concat_cols[2:]:
      df[new_col] = df[new_col] + space + df[concat_col]

  # -----------------------------------------------------
  # Convert a numeric column to a categorical column based on ranges defined by bins
  # -----------------------------------------------------
  @df_process(save_res=True, use_fd=False)
  def bin_num (self, df, col, bin_col, bins, labels):
    df[bin_col] = pd.cut(df[col], bins=bins, labels=labels)

  # PROCESSORs
  # ===============================================================

  # ----------------------------
  # Convert all categorical variables to numeric category codes, using the 
  # Pandas Categorical data type. Categorical variables get converted to
  # Categorical columns at the beginning before Splitting. That way all the
  # category values are based on all data rows, not just the training rows.
  # If this isn't done, then values which occur only in the validation rows
  # would be given an 'unknown' category code.
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def to_cat(self, df, convert_state, in_train, cols=None, ordinal_vals={}, **kwargs):
    cols = cols if (cols is not None) else self._categorical_cols()

    for col in cols:
      if (in_train):
        if not is_categorical_dtype(df[col]):
          # Convert to Categorical data type if it isn't already. All categorical
          # variables have already been converted at the beginning itself, so it is
          # only the '_na' variables that were newly added that get converted here.
          df[col] = df[col].astype('category')

        # Set the category value using the given order, if the values are 
        # ordinal ie. they have an intrinsic order
        if (col in ordinal_vals):
          vals = ordinal_vals[col]
          df[col].cat.set_categories(vals, ordered=True, inplace=True)

        # Save away the category names for use with validation data
        convert_state[f'Category_{col}'] = df[col].cat.categories

        # Get the category codes for training data. Pandas category codes start 
        # from 0, and -1 is used for NaN values. So we add 1 to all codes which means 
        # that NaN get a 0 value, and all other categories get shifted by 1.
        df[col] = df[col].cat.codes + 1
      else:
        # For validation and test data, use the same categories that were saved for
        # training data. Set the ordered flag if the column is ordinal.
        ordered = (col in ordinal_vals)
        cat_col = pd.Categorical(df[col], categories=convert_state[f'Category_{col}'], ordered=ordered)
        df[col] = cat_col.codes + 1

  # ----------------------------
  # Replace any '+- inf' values by NaN
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def remove_inf(self, df, **kwargs):
    continuous_cols = self._continuous_cols()
    df[continuous_cols] = df[continuous_cols].replace([np.inf, -np.inf], np.nan)

  # ----------------------------
  # Check columns for count of missing values. Optionally add a flag column for each
  # column with missing values
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def check_missing(self, df, convert_state, in_train, cols=None, missing_thresh=0.5, add_flag=False, **kwargs):
    if (cols is None):
      cols = self._continuous_cols()

    # %age of missing values > threshold
    missing_pct = df[cols].isna().sum() / len(df)
    high_missing = missing_pct[missing_pct > missing_thresh].to_dict()
    for col, pct in high_missing.items():
      print (f'Column {col} has {pct} percent missing rows. In Train: {in_train}')

    # Check if flag columns need to be added
    if (not add_flag):
      return

    if (in_train):
      # For training, find all columns with missing data and save the list
      add_cols = missing_pct[missing_pct > 0].index.tolist()
      convert_state[f'NAColumns'] = add_cols
    else:
      # For validation use the list of missing columns saved from training 
      add_cols = convert_state[f'NAColumns']

    # Add an extra column with a flag indicating that a value was 
    # missing, before being filled. Add those extra columns to the list of
    # categorical columns.
    for col in add_cols:
      na_col = col + '_na'
      df[na_col] = df[col].isna()
      self.fd['nominal'].append(na_col)

  # ----------------------------
  # Helper function to impute values for missing column values
  # ----------------------------
  def _impute_values(self, df, cols, strategy, values=None):
    imp_df = df[cols]

    # Compute the mean, median or mode depending on the strategy. If strategy
    # is 'constant', the values are passed in so there is nothing to compute
    if (strategy == 'mode'):
      # If multiple values have the same count, mode() returns multiple rows.
      # So .iloc[0] picks the first row
      fill_dict = imp_df.mode().iloc[0].to_dict()
    elif (strategy == 'median'):
      fill_dict = imp_df.median().to_dict()
    elif (strategy == 'mean'):
      fill_dict = imp_df.mean().to_dict()
    elif (strategy == 'constant'):
      fill_dict = {k:v for k, v in zip(cols, values)}

    # Use the computed (or passed in) values to fill all missing NaNs
    df.fillna(fill_dict, inplace=True)

    # Check that there are no missing values left
    assert(df[cols].isna().sum().sum() == 0)
    return list(fill_dict.values())

  # ----------------------------
  # Fill in missing column values
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def fill_missing(self, df, convert_state, in_train, cols=None, strategy='median', fill_values=None, **kwargs):
    assert(strategy in ['mean', 'median', 'mode', 'constant'])
    assert(not((strategy == 'constant') and (fill_values is None)))

    fill_cat = False    # For now, don't fill categorical columns
    if (in_train):
      if (cols is None):
        continuous_cols = self._continuous_cols()
        continuous_values = self._impute_values(df, continuous_cols, strategy, fill_values)
        cols, values = continuous_cols, continuous_values

        if (fill_cat):
          # Get the categorical columns and impute the mode values for them
          categorical_cols = self._categorical_cols()
          categorical_values = self._impute_values(df, categorical_cols, 'mode')
          cols += categorical_cols
          values += categorical_values
      else:
        values = self._impute_values(df, cols, strategy, fill_values)

      # Save the values that were used to fill missing NaNs, so that we can use
      # them for validation data 
      for col, val in zip(cols, values):
        convert_state[f'Missing_{col}'] = val
    
    else:
      # For validation data, fill with the same values that were used for the 
      # training data

      if (cols is None):
        cols = self._continuous_cols()
        if (fill_cat):
          cols += self._categorical_cols()

      fill_values = [convert_state[f'Missing_{col}'] for col in cols]
      _ = self._impute_values(df, cols, strategy='constant', values=fill_values)

  # ----------------------------
  # Normalisation
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def normalise(self, df, convert_state, in_train, cols=None, **kwargs):

    # If no columns are given, normalise all continuous columns
    if (cols is None):
      cols = self._continuous_cols()

    for col in cols:
      if (in_train):
        # Calculate the mean and std for training data and save it for use
        # during validation.
        # Since we have earlier optimised the DataFrame memory by changing columns 
        # to the lightest data type that encompasses the min-max range, but that 
        # could cause the mean and std to overflow. So we temporarily treat them
        # as doubles for the mean/std calculation by using .astype('float64')
        convert_state[f'Mean_{col}'] = df[col].astype('float64').mean()
        convert_state[f'Std_{col}'] = df[col].astype('float64').std()

      # Use the saved mean/std to normalise for both training data and
      # validation data
      col_mean = convert_state[f'Mean_{col}']
      col_std = convert_state[f'Std_{col}']
      assert (np.isfinite(col_mean) and np.isfinite(col_std))

      df[col] = (df[col].astype('float64') - col_mean) / (1e-7 + col_std)

  # ----------------------------
  # Return a numpy array of all the column values for the i-th row
  # ----------------------------
  @df_process(save_res=True, use_fd=False)
  def to_np(self, df, **kwargs):
    cat_cols, cont_cols = self._categorical_cols(), self._continuous_cols()
    all_cols = cat_cols + cont_cols + self._target_cols()

    # Check that there are no NaNs
    assert(df[cont_cols].isna().sum().sum() == 0)
    #assert(np.isfinite(df[cont_cols]).all().all())

    # List of columns values for all columns. Make sure that all the
    # categorical columns are first, followed by all the continuous columns
    vals = [df[col].values for col in all_cols if col in df.columns]

    # Convert list to numpy array
    arr = np.stack(vals, axis=1)

    # Convert to float if it isn't already
    if (not np.issubdtype(arr.dtype, float)):
      arr = arr.astype('float32')

    self.arr = arr

  # ----------------------------
  # Compare two Dataframes for equality, for debugging purposes
  # ----------------------------
  def compare_df(self, df1, df2, name=''):
    # Compare the two dfs column-wise
    diff_df = (df1 == df2).all(0)
    # Get a list of columns which are different
    diff_cols = diff_df[~diff_df].index.tolist()

    # Since NaN cannot be compared with another NaN, columns which
    # have NaN values are marked as being different. So we filter
    # out columns which differ only in the NaN values
    unmatched_cols=[]
    for col in diff_cols:
      # Compare the non-NaN values in both columns
      if (df1[df1[col].notna()][col] == df2[df2[col].notna()][col]).all(0):
        # Columns are same
        pass
      else:
        # Columns are different
        unmatched_cols.append(col)

    # Get the rows which are different in the unmatched columns
    if (unmatched_cols):
      um_df1, um_df2 = self._unmatched_rows(df1, df2, unmatched_cols)
      print (f'Unmatched {name}', unmatched_cols)
    else:
      um_df1, um_df2 = None, None
      print (f'Matched {name}')

    return [(unmatched_cols, um_df1, um_df2)]

  # ----------------------------
  # Supporting function, returns the unmatched rows between two Dataframes
  # ----------------------------
  def _unmatched_rows(self, df1, df2, unmatched_cols):
    unmatch_mask = (df1[unmatched_cols] != df2[unmatched_cols]).any(1)
    um_df1 = df1[unmatch_mask][unmatched_cols]
    um_df2 = df2[unmatch_mask][unmatched_cols]
    print ('Num unmatched', len(um_df1))
    return (um_df1, um_df2)

  # ----------------------------
  # Compare two pairs of np arrays (ie. X1 to X2, and Y1 to Y2) for equality, for 
  # debugging purposes
  # ----------------------------
  def compare_arr(self, x_arr1, x_arr2, y_arr1, y_arr2, n_cat):
    # Compare shapes
    assert(x_arr1.shape == x_arr2.shape)
    assert(y_arr1.shape == y_arr2.shape)

    # Compare all categorical columns
    diff_cat = (x_arr1[:, :n_cat] == x_arr2[:, :n_cat]).all(0)
    assert(diff_cat.all())

    # Compare all continuous columns
    diff_cont = (x_arr1[:, n_cat:] == x_arr2[:, n_cat:]).all(0)
    assert(diff_cont.all())

    # Compare the target columns
    diff_tgt = (y_arr1 == y_arr2).all()
    assert(diff_tgt.all())

  # EDA and Metadata functionality
  # ===============================================================
  
  # What we want
  # - Names of all columns with their pandas data types
  # - Logical type of each column ie. Categorical Ordinal, Categorical Nominal, Continuous, Target, Primary Key
  # - Number of rows
  # - Categorical columns distribution - number of unique values, list of unique values with their counts
  # - Continuous columns distribution - histogram or density curve
  # - For each column - number and %age of missing values
  # - describe() stats
  #
  # - Bivariate distribution of each column with the Target column
  #
  # - For time series - multi-index on time and primary key. Then plot time on X-axis and
  #    Target value on y-axis, with separate lines for each value of the primary key. Any
  #    event occurrences can also be marked on the timeline.

  # ----------------------------
  # Make a first guess about the feature type of each column.
  # We are given 'mds' which is the output of df.describe()
  # ----------------------------
  def _feature_types(self, mds):
    # Initialise all columns to None
    mds['col_type'] = None

    # Mark columns, where > 30% rows are missing, as secondary
    miss_mask = (mds['col_type'].isna()) & (mds['% miss'] > 30)
    mds.loc[miss_mask, 'col_type'] = 'secondary'

    # Mark columns, with finite unique values, as categorical
    if ('unique' in mds.columns):
      cat_mask = (mds['col_type'].isna()) & mds.loc[:, 'unique'].notnull()
      mds.loc[cat_mask, 'col_type'] = 'categorical'

    # Mark columns, where > 50% rows have the same mode value, as secondary
    sec_mask = (mds['col_type'].isna()) & (mds['mode %'] > 50)
    mds.loc[sec_mask, 'col_type'] = 'secondary'

    # Mark columns, which are either ints or floats, as numeric
    num_mask = (mds['col_type'].isna()) & (mds['dtype'].astype('str').str.startswith('float') | mds['dtype'].astype('str').str.startswith('int'))
    mds.loc[num_mask, 'col_type'] = 'continuous'

  # ----------------------------
  # Summarise metadata about all the columns in the df
  # ----------------------------
  @df_process(save_res=False, use_fd=False)
  def metadata_summary (self, df):
    # Each DataFrame column is a column in the describe() output. 
    mds = df.describe(include = 'all')
    df_len = len(df)

    # Datatypes and total counts
    mds.loc['dtype'] = df.dtypes
    mds.loc['size'] = df_len

    # Most common value and % occurrence of that value
    for col in mds.columns:
      vc = df[col].value_counts()
      mds.loc['mode', col] = vc.idxmax()
      mds.loc['mode %', col] = round(vc.max() / df_len, 3) * 100

    # Count and % of missing values
    # !!!!!! Different ways of calculating, keep only one
    mds.loc['missing'] = len(df.index) - df.count()
    mds.loc['% count'] = df.isnull().mean() * 100
    mds.loc['% missing'] = df.isnull().sum()/df_len*100
    mds.loc['% miss'] = df.isna().mean().round(4) * 100

    # Each DataFrame column now has a row in the 'mds' summary
    mds = mds.transpose()

    # Fill in the feature types for continuous, categorical etc.
    self._feature_types(mds)

    # For some reason, some Dataframes don't have 'unique' or 'min', 'mean' etc in their describe output
    # Include 'unique' in the output if present
    unique_cols = []
    if ('unique' in mds.columns):
      unique_cols = ['unique']

    # Include 'min', 'mean' etc if present
    stat_cols = []
    if ('mean' in mds.columns):
      stat_cols = ['min', 'mean', '50%', 'max']

    cols = ['dtype', 'size', 'missing', '% miss', 'mode', 'mode %', 'col_type'] + unique_cols + stat_cols
    mds = mds[cols]
    return mds

  # ----------------------------
  # Calculate the distribution of the 'Many' values in the right hand side of a
  # One-Many (or One-One) Foreign Key relationship.
  #
  # For example, if the right-hand DataFrame has key columns as:
  # [fk1, fk2, pk1, pk2, pk3] 
  #    where [fk1, fk2] are foreign keys to the left-hand DataFrame
  # 
  # For each unique combination of [fk1, fk2], how many unique combinations of
  # [pk1, pk2, pk3] are there? That is the fanout or cardinality of the 'Many' 
  # side of the relationship.
  # For one combination of [fk1, fk2], we could have a fanout of 3, say. In other
  #    words, there are 3 combinations of [pk1, pk2, pk3] for that [fk1, fk2]
  # Similarly, other combinations of [fk1, fk2] would have other fanout values.
  #
  # Here, we calculate the distribution of those fanout values ie. how many 
  # occurrences of each fanout value do we have.
  # ----------------------------
  @df_process(save_res=False, use_fd=True)
  def fk_relations_fan(self, df, fd, levels=0, **kwargs):
    key_cols = fd['key']

    lev = levels + 1
    lev_col = key_cols[:lev]
    size_col = key_cols[lev]
    df_fan = df.groupby(lev_col)[size_col].nunique().sort_values(ascending=False).reset_index()

    return (df_fan, size_col)

  # ----------------------------
  # Select some sample rows for exploratory analysis from two dfs which have a 
  # One-Many (or One-One) Foreign Key relationship. Since the data can be large
  # we fetch only a subset where the 'Many' side of the relationship has a count
  # of 'size_val'. Also since the foreign key could have multiple index columns,
  # we specify the 'levels' at which we take the count.
  #
  # !!!!!! NB: We still need to handle the case where key_cols1 has multiple columns
  # and so vals2_match is a tuple and gb2_match.values is a dataframe not a series
  # ----------------------------
  @df_process(save_res=False, use_fd=True)
  def fk_relations(self, df, right_df, fd, right_fd, size_val=3, max_rows=10, levels=0, **kwargs):
    left_df, left_fd = df, fd
    df1, df2 = left_df, right_df
    key_cols1 = left_fd['key']
    key_cols2 = right_fd['key']
    cols1 = left_fd['primary']
    cols2 = right_fd['primary']

    # Group at the given level, and get the size (ie. number of rows) of each group
    lev = levels + 1
    lev_col = key_cols2[:lev]
    size_col = key_cols2[lev]
    gb2 = df2.groupby(lev_col)[size_col].nunique().reset_index()

    # Get the groups that have the given group size 'size_val'
    # Then, truncate if we get more than 'max_rows' groups
    # Then, get the key IDs of the matching groups
    len_lev_col = len(lev_col)
    gb2_match = gb2[gb2[size_col] == size_val].iloc[:max_rows, :len_lev_col]
    if (len_lev_col > 1):
      vals2_match = [tuple(val) for val in gb2_match.values]
    else:
      vals2_match = gb2_match.values[:, 0]

    # Now use those matching key IDs to select a subset of rows from the dataframe
    df2_sub = df2.set_index(lev_col).loc[vals2_match, key_cols2[lev:]+cols2].reset_index()

    keys_intersection = list(set(key_cols1) & set(key_cols2))
    df1_sub = df1[key_cols1+cols1]
    merge_df = df1_sub.merge(df2_sub, how='inner', left_on=keys_intersection, right_on=keys_intersection, suffixes=('_l', '_r'))

    # Select the columns we need
    keys_union = key_cols1 + [key2 for key2 in key_cols2 if (key2 not in key_cols1)]
    merge_df = merge_df.set_index(keys_union).sort_values(keys_union)
    return merge_df

  # ----------------------------
  # Reduce memory usage by optimising the data types of each column
  # ----------------------------
  @staticmethod
  def reduce_mem(df, name):
    start_mem = df.memory_usage().sum() / 1024**2
    
    len_df = len(df.index)
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            # Find the min and max range of values in each column and then use
            # the smallest data type that can fit those values.
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32)                    
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                    df[col] = df[col].astype(np.uint64)

            elif str(col_type)[:5] == 'float':
                # Commented out because arithmetic with float16 results in
                # non-deterministic results due to floating point precision.
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

        elif (False):
            # Commented out because categorical columns causes huge performance issues
            # with rollups - crosstab, pivot_tables
            # If the unique values are below a threshold, convert to categorical
            assert(False)
            unique_val = df[col].nunique()
            if ((unique_val < 500) and (unique_val / len_df < 0.1)):
              df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    mem_reduction_pct = (100 * (start_mem - end_mem) / start_mem)
    print(f'{name} memory reduced by {mem_reduction_pct:.1f}% from {start_mem:.2f} MB to {end_mem:.2f} MB')
    
    return df

  # UNUSED METHODS
  # They are alternate implementations for functionality with better implementations
  # above. But they are left here as examples of different ways to achieve the
  # functionality
  # ===============================================================

  # ----------------------------
  # Alternate implementation for getting rolled up counts of categorical columns
  # instead of using crosstab (but doesn't do normalise)
  # ----------------------------
  def WAIT_roll_cat_alternate(df):
    df_count = df.pivot_table(index=index, columns=[col], values=count_col, aggfunc='count', fill_value=0)
    col_headers = df_count.columns.levels[1]
    df_count.columns = [f'{col_headers.name}_{val}' for val in col_headers.to_list()]

  # ----------------------------
  # Alternate implementation to flatten multi-level column index
  # ----------------------------
  @staticmethod
  def WAIT_flatten_col_basic(df):
      levels = df.columns.nlevels
      if (levels == 2):
        df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
      elif (levels == 3):
        df.columns = df.columns.map('{0[0]}_{0[1]}_{0[2]}'.format)

  # ----------------------------
  # Alternate implementation to flatten multi-level column index
  # ----------------------------
  @staticmethod
  def WAIT_flatten_col_names(df):
      col_names=[]
      for col, agg_fn in df.columns:
        col_names.append(f'{col}_{agg_fn}')
      df.columns = col_names

  # ----------------------------
  # Alternate implementation to flatten multi-level column index
  # ----------------------------
  @staticmethod
  def WAIT_flatten_col_alternate(df):
      levels = df.columns.nlevels
      if (levels > 1):
        s = '_'.join([f'{{0[{i}]}}' for i in range(levels)])
        df.columns = df.columns.map(s.format)

  # ----------------------------
  # Transform - not used right now, only for testing
  # ----------------------------
  def test_xform(self, obj):
    return obj

  # ----------------------------
  # Min-Max Scaling - not used right now
  # ----------------------------
  def minmax_scaling (df, cols):
      # Get np array for scaling. 
      # Adjust to 2D if cols contains only a single element
      x = df[cols].values
      if (x.ndim == 1):
          x = x.reshape(-1,1)
      
      min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1)) 
      x_after_min_max_scaler = min_max_scaler.fit_transform(x)
      df[cols] = x_after_min_max_scaler

  # ----------------------------
  # Helper function to impute values for missing column values using sklearn's
  # SimpleImputer. It is not used now because of extreme performance issues for
  # strategy 'most_frequent' which computes the mode(). We now use Pandas methods
  # to impute these values.
  # ----------------------------
  from sklearn.impute import SimpleImputer
  def _sk_impute_values(self, df, cols, strategy, fill_value=None):
    imp_df = df[cols]
    imr = SimpleImputer(missing_values=np.nan, strategy=strategy, fill_value=fill_value)
    imr = imr.fit(imp_df)
    df[imp_df.columns] = imr.transform(imp_df.values)
    return (imr.statistics_.tolist())

#### Tabular Data Bundle

In [ ]:
#export

class FeatureList():
  def __init__(self):
    self.fss = {}

  def add_fs(self, fs):
    self.fss[fs.fs_name] = fs

  def get_cols(self, fs_names=None):
    fs_names = fs_names if fs_names else self.fss.keys()

    cat_cont = [self.fss[fs_name].get_cols() for fs_name in fs_names]
    cats, conts = zip(*cat_cont)
    
    # Flatten the list of lists
    cat_cols = [cat_col for cat in cats for cat_col in cat]
    # Remove duplicates while preserving the order
    # 'Seen' is a set of all items seen so far. As we go through every item we add it
    # to the set if it isn't there already. 
    # seen.add() always returns true, so basically the condition says to select each
    # item 'x' if it is not already in the 'seen' set.
    # See https://stackoverflow.com/questions/45746312/one-liner-to-remove-duplicates-keep-ordering-of-list
    seen = set()
    cat_cols = [x for x in cat_cols if not (x in seen or seen.add(x))]

    # Flatten the list of lists
    cont_cols = [cont_col for cont in conts for cont_col in cont]
    # Remove duplicates while preserving the order
    seen = set()
    cont_cols = [x for x in cont_cols if not (x in seen or seen.add(x))]

    return (cat_cols, cont_cols)

class FeatureSet():
  def __init__(self, fs_name, cat_cols, cont_cols):
    self.fs_name = fs_name
    self.cat_cols, self.cont_cols = cat_cols, cont_cols

  def get_cols(self):
    return (self.cat_cols, self.cont_cols)

In [ ]:
#export

#----------------------------------------------------
# Tabular data preparation pipeline
#----------------------------------------------------
class TabularDataBundle(DataBundle):
  def __init__(self, csv_path, test_csv_path=None, related_csv_paths=None, prepare_fn=None, 
               test_data=False, missing_thresh=0.5, add_flag=False, ordinal_vals={}, **kwargs):
    self.tgt_range = None

    print ('--------- Tabular Home Credit DataBundle init', csv_path, test_csv_path, related_csv_paths)

    # Load all rows from the given CSV file
    # Split randomly based on a percentage ratio for training and validation
    # 'x' items are taken from 'Reviews' column as text sentences and
    # 'y' labels are taken from 'Sentiments' column as class name labels
    # Convert the 'x' items from Sentences to Words to Word Ids
    # Convert the 'y' items from Class Names to Class Ids

    # load takes a processor function which CSVItemContainer calls after creating the Tabular List. This function will be on the DataBundle.
    # This processor does all data cleaning, merging of related dfs, feature engineering etc. It ends with a Tabular List that has a single
    # df only, with items matching the length, and with a feature dict. Until that point, the 'items' are not used at all during the
    # processor func. After this point the Tabular List with df is fully ready for particpating in the rest of the Data Bundle process. So
    # it can get split, and x/y extracted.
    #
    # The TabularDataBundle base class defines a generic processor func, which all child DataBundles inherit. The base function creates
    # items on the ItemList which match the df length, and it also sets the feature dict including all columns created during feature
    # engineering in the processor.
    #
    # Load also takes an optional 'test_csv' argument for loading the test data.
    # The split function splits the train file into train and valid and takes the test file as is. So all the split functions take either 
    # a 'test_ratio' or a 'test_data' flag. The test data flag points to an attribute on the itemlist which contains the test data??
    #
    # The extract_colval is used to extract column data as a list, do some processing on it and convert it to another target class. In the
    # case of Tabular, we'll use a select_col function which will select the non target and target columns and end with the same Tabular
    # target class.
    #
    # The convert_x_params can then be quite generic and cover just the to_cat, fill_missing and normalise steps.
    #
    # The Tabular List methods are therefore groupable into three buckets:
    #   1. Those needed for EDA (use feature dict)
    #   2. Those needed for pre-processing (don't use feature dict)
    #   3. Those needed for convert_ing -- these are the only ones which take convert_state, in_train etc. (use feature dict)
    #
    # The workflow then is like this:
    #   1. Manually call load on the DataBundle without any processor
    #   2. Then do the data exploration EDA steps - metadata, relation, distribution graphs - these are all read-only.
    #   2a.  NB: After metadata, the feature dict can get set and gets used for the remaining EDA operations. This is just the initial feature dict, which
    #       will change later after processing once all new features are added.
    #   3. Now run the full-fledged DataBundle with a processor. Incrementally, during dev, you can keep adding more and more steps one by one to the processor.
    #       After each step, you can do any output of sample(), or graphing that you want to. As you are iterating you can keep refining the feature dict
    #       as you go, if you want to.
    #   4. Once you are happy with all the processor steps and it is ready, then define the final feature dict, and you can then just continue running 
    #       the full-fledged DataBundle just as you did in #3 above and feed the DataLoader to the model.

    load_params = {'source': CSVItemContainer, 'target_cls': TabularItemList, 'csv_path': csv_path, 'test_csv_path': test_csv_path, 'related_csv_paths': related_csv_paths, 'prepare_fn': prepare_fn}
    split_params = {'split_procedure': 'split_sequential', 'train_ratio': 0.8, 'valid_ratio': 0.2, 'test_data': test_data}
    extract_x_params = {'extract_procedure': 'extract_type', 'target_cls': TabularItemList, 'cols_type': 'cat_cont'}
    extract_y_params = {'extract_procedure': 'extract_type', 'target_cls': TabularItemList, 'cols_type': 'tgt'}

    convert_x_params = [
         {'target_cls': TabularItemList, 'convert_procedure': 'remove_inf'},
         {'target_cls': TabularItemList, 'convert_procedure': 'check_missing', 'missing_thresh': missing_thresh, 'add_flag': add_flag},
         {'target_cls': TabularItemList, 'convert_procedure': 'fill_missing', 'cols': None, 'strategy': 'median'},
         {'target_cls': TabularItemList, 'convert_procedure': 'normalise', 'cols': None},
         {'target_cls': TabularItemList, 'convert_procedure': 'to_cat', 'cols': None, 'ordinal_vals': ordinal_vals},
         {'target_cls': TabularItemList, 'convert_procedure': 'to_np'},
    ]
    convert_y_params = [
         {'target_cls': TabularItemList, 'convert_procedure': 'to_np'}
    ]
    xform_x_params = [
#        {'xform_procedure': 'test_xform'}, 
    ]
    dl_params = (
        {'bs': 64, 'sampler_fn': RandomSampler},        # for training
        {'bs': 64, 'sampler_fn': SequentialSampler}     # for valid/test
    )
    self.display_params = {
        'layout_procedure': 'display_texts'
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=xform_x_params, dl_params=dl_params)

  # ----------------------------
  # This is the final step of the 'Prepare' processing. By this time, all data
  # cleaning and feature generation is done, and all related dfs have been merged
  # into the main df. We are given the final set of continuous and categorical
  # variables. So here, the final ItemList is re-created with only the main
  # df and with just the columns that are required. The Feature Dict is also defined
  # based on the given variables.
  # ----------------------------
  def prepare(self, il, fl, tgt_cols):
    # The df is now fully prepared after all the data processing and wrangling.

    cat_cols, cont_cols = fl.get_cols()

    # Convert categorical variables to Pandas Categorical data type. This ensures
    # that the category values are based on both training and validation rows before 
    # splitting.
    il.df[cat_cols] = il.df[cat_cols].astype('category')

    cols_all = cat_cols + cont_cols + tgt_cols
    il.df = il.df[cols_all].reset_index(drop=True)

    features_dict = {
        'nominal'     : cat_cols,
        'continuous'  : cont_cols,
        'target'      : tgt_cols,
    }
    il.add_features_dict(features_dict, df_name=None, mds=None)

    # Reset the ItemList by re-creating it with the finished df and without the 
    # related dfs.
    il = il.__class__(None, il.df)
    return il

  # ----------------------------
  # Get count of continuous columns and category names for categorical columns, as
  # required to build the model
  # ----------------------------
  def col_szs(self):
    continuous_cols = self.train_ds.x._continuous_cols()
    n_cont = len(continuous_cols)

    cat_cols = self.train_ds.x._categorical_cols()
    cat_szs = [len(self.convert_state_x[f'Category_{cat_col}']) + 1 for cat_col in cat_cols]

    n_tgt = len(self.train_ds.y._target_cols())

    return n_cont, cat_szs, n_tgt, self.tgt_range

  # ----------------------------
  # Display metadata about each column in the DataFrame
  # ----------------------------
  def display_metadata(self, il, df_name=None):
    mds = il.metadata_summary(df_name=df_name)
    DisplayData.display_summary(mds)
    return mds

  # ----------------------------
  # Show one-many relationship between two DataFrames and display sample rows 
  # ----------------------------
  def display_relation(self, il, right_df_name, df_name=None, size_val=3, max_rows=10, levels=0):
    fan_df, x = il.fk_relations_fan(df_name=right_df_name, levels=0)

    merge_df = il.fk_relations(df_name=df_name, right_df_name=right_df_name, size_val=size_val, max_rows=max_rows, levels=levels)
    DisplayData.display_rel(fan_df, x, merge_df)

  # ----------------------------
  # Compare two DataBundles for equality, for debugging purposes. 
  # ----------------------------
  def compare_db(self, db1, db2):
    all_il1 = getattr(db1, 'all_rows', None)
    all_il2 = getattr(db1, 'all_rows', None)
    if (all_il1 and all_il2):
      # Compare 'load' df if present
      all_df1 = all_il1.df
      all_df2 = all_il2.df
      all_unmatched = all_il1.compare_df(all_df1, all_df2, name='All Rows')
      return all_unmatched
    else:
      # Otherwise compare 'post_load' df and arr
      tr_unmatcheds = self._compare_ds(db1.train_ds, db2.train_ds, name='Training')
      val_unmatcheds = self._compare_ds(db1.valid_ds, db2.valid_ds, name='Valid')
      return (tr_unmatcheds + val_unmatcheds)

  # ----------------------------
  # Compare two Datasets for equality, for debugging purposes
  # ----------------------------
  def _compare_ds(self, ds1, ds2, name):
    x_il1, y_il1, x_il2, y_il2 = ds1.x, ds1.y, ds2.x, ds2.y
    x_df1, y_df1, x_arr1, y_arr1 = x_il1.df, y_il1.df, x_il1.arr, y_il1.arr
    x_df2, y_df2, x_arr2, y_arr2 = x_il2.df, y_il2.df, x_il2.arr, y_il2.arr
    
    assert(x_il1._categorical_cols() == x_il2._categorical_cols())
    assert(x_il1._continuous_cols() == x_il2._continuous_cols())
    n_cat = len(x_il1._categorical_cols())

    # Compare training df
    x_unmatched = x_il1.compare_df(x_df1, x_df2, name=f'{name} X')
    y_unmatched = y_il1.compare_df(y_df1, y_df2, name=f'{name} Y')

    # Compare training arr
    x_il1.compare_arr(x_arr1, x_arr2, y_arr1, y_arr2, n_cat)

    return (x_unmatched + y_unmatched)

  # ----------------------------
  # Helper function for making subsets which should be overridden by the subclass
  # to define parameters for related dfs
  # ----------------------------
  @classmethod
  def subset_related(cls):
    idx_related = None
    merge_fn = None
    return (idx_related, merge_fn)

  # ----------------------------
  # Create smaller subsets of the main and test dfs, along with matching rows from
  # all the related dfs
  #
  # TODO: !!!!!!!!!! Currently, this assumes that there is a single merge_fn which merges
  # all related files together at once. However such a function may not exist in most cases because
  # most of the time, related files will get aggregated and rolled up into the main file. Also
  # it may not be possible to merge all related files into the main file at once. Some related
  # files may be hierarchically related at a second or deeper level, and will get merged into 
  # another related file first using a different primary key, and then that file gets merged 
  # to the main file eg. in the Home Credit application, bureau_balance first gets merged into
  # bureau, when then gets merged into the main file.
  #
  # So basically, the approach we need to use here is for the merge to happen one related file
  # at a time. So each related file might have its own separate merge function. So one
  # related file gets merged and a subset gets created for it, then you loop to the next one
  # and so on. Note that it could be more than one file as well, like in the hierarchical 
  # example above, you might merge two related files into the main file together. The real point
  # is that it will not be a merge_all.
  # ----------------------------
  def make_subset(self, idx_related, merge_fn, subset_dir, subset_sz=100):
    il = self.process(steps=['load'])

    main_path = self.load_params['csv_path']
    test_path = self.load_params['test_csv_path']

    # Keep only the first 'subset_sz' of main df and test df, if it is present
    # Save them to subset files
    subset_dir.mkdir(exist_ok=True)
    il.df = il.df.iloc[:subset_sz]
    il.df.to_csv(subset_dir / main_path.name, index=False)
    if (test_path):
      test_df = il.test_rows.df.iloc[:subset_sz]
      test_df.to_csv(subset_dir / test_path.name, index=False)
      il.df = il.df.append(test_df)

    # Now find the rows from all the related dfs which would get joined to
    # the subset rows

    if (not idx_related):
      return

    # Add the index as its own column named 'idx_<abbr>' so that we
    # can track each row number from each related df
    for df_name, idx_name in idx_related.items():
      il.add_index_col(df_name=df_name, idx_col='idx_' + idx_name)

    # Join all the dfs into a single df
    merge_fn(il)

    # Since the index columns are also merged, they can be used to identify which row
    # numbers from the original related dfs were joined together. Note that if the 
    # same df was joined more than once, the index column will occur multiple times, each
    # with a different suffix.

    # Create an index df by selecting just the index columns from the merged df
    idx_names = [col for col in il.df.columns if col.startswith('idx_')]
    idx_df = il.df[idx_names].copy()

    # Now discard the merged df and reload the original dfs once again
    il = self.process(steps=['load'])

    # Use the index df to pull out the corresponding row numbers from each original df
    # Save those rows to the subset file
    for df_name, idx_name in idx_related.items():
      df = il._get_df(df_name)
      # Get row ids from all the index columns for this original df, since the index column might 
      # occur multiple times
      idxs = [idx_df[col].tolist() for col in idx_df.columns if col.startswith('idx_' + idx_name)]
      # Flatten the list of sublists, and remove duplicates using set()
      idxs = set([idx for sublist in idxs for idx in sublist])
      # Use the row ids to get the corresponding rows from the df
      df = df.loc[idxs]
      # Save the subset file
      df.to_csv(subset_dir / (df_name + '.csv'), index=False)


In [ ]:
# -------------------------
# WORKFLOW
# -------------------------
# Download data
# Main File only
  # EDA - MDS (TabularBaseDB.load)
  # EDA - Distrib
  # Cleaning - Prepare (MyTabularDB.load)
  # Subset - 10 batches
  # Convert - Continuous columns only - check Fill Missing and Normalise
  # Run Model - DTR, Recorder, Metrics, Loss
  # Convert - Add Categorical columns - check To Cat
  # Feature Engg - Prepare
  # Subset - 100 batches
# Related - one file at a time - 100 batches
  # EDA - MDS, Distrib, Related
  # Cleaning
  # Subset - add Related
  # Rollup - Prepare
  # Run Model
# Feature Engg - Prepare
# Subset - 1000 batches

# -------------------------
# DATABUNDLE PROCESS and MODEL
# -------------------------
# Load and Prepare (Save/Load Pickle)
# Process - Split, Extract and Convert (Save/Load Pickle)
# Model (Save/Load Weights)

# -------------------------
# COMPARE
# -------------------------
# Compare Prepared data (df)
# Compare Processed data (train dfx/dfy and val dfx/dfy)
# Compare Np Array data (np array train x/y and val x/y)
# Compare Model results (results df)

# -------------------------
# OBJECTS
# -------------------------
# Tabular ItemList
# Tabular DataBundle
# Arch - model, hook layer groups, discrim lr param groups, load/save weights
# App - pre-process data, load data, create arch, run train, run predict
# Trainer - runs training loop
# Optimiser (updates weights using gradients and hyperparam) and HyperParam (varies hyperparams in two dimensions across param groups (discrim) and across batches (scheduler))
# Metrics - compute metrics
# Hooks and Debug Callback - probes for capturing internal layer values
# Debug Tracker - collects all Debug/Hook values, Metrics, HyperParams.
# Reporter/Presenter/Recorder - outputs to DF, Tensorboard and Progress. DF outputs to Seaborn/Matplotlib

   # The workflow then is like this:
    #   1. Manually call load on the DataBundle without any processor
    #   2. Then do the data exploration EDA steps - metadata, relation, distribution graphs - these are all read-only.
    #   2a.  NB: After metadata, the feature dict can get set and gets used for the remaining EDA operations. This is just the initial feature dict, which
    #       will change later after processing once all new features are added.
    #   3. Now run the full-fledged DataBundle with a processor. Incrementally, during dev, you can keep adding more and more steps one by one to the processor.
    #       After each step, you can do any output of sample(), or graphing that you want to. As you are iterating you can keep refining the feature dict
    #       as you go, if you want to.
    #   4. Once you are happy with all the processor steps and it is ready, then define the final feature dict, and you can then just continue running 
    #       the full-fledged DataBundle just as you did in #3 above and feed the DataLoader to the model.

      # Normal Workflow - Break down the problem into smaller pieces
     # Run with base-data ie only 'train.csv' columns with no related.
     # Subset with say 10 batches - ie. 640 rows. Or with 10000 rows?
     # Test with only continuous columns first
     # Then add categorical columns
     # Test that fill missing and normalise give us correct results
     # Then add rollup of related, but only one table at first. Then slowly add more tables.
     # Then make a subset of Small (10/100 batches), then Mid (1000) and Full(10000)

#### NpItemList

In [ ]:
#----------------------------------------------------
# ItemList for a Numpy array. Each row is an item. Since the actual data 
# is stored in the array, the items themselves are just a range of numbers 
# from 0 to 'array_length - 1'.
#----------------------------------------------------
class NpItemList(ItemList):
  # ----------------------------
  # ----------------------------
  def __init__(self, items, arr, **kwargs):
    # If present, 'items' should be the same length as the array
    assert (len(items) == arr.shape[0])
    self.arr = arr
    super().__init__(items)
    self._copy_new.append('arr')

  # ----------------------------
  # To get the actual data object, we use the item as an index into the
  # numpy array's rows. Therefore item #i corresponds to array row #i.
  # ----------------------------
  def _get_object(self, i):
    return self.arr[i]


#### To Be Sorted - EDA and other Utils

In [ ]:
# ------- THESE ARE NOT USED RIGHT NOW -----------------

# CONVERT THE DATA
# ===============================================================

# -----------------------------------------------------
# Label encode categorical columns
# -----------------------------------------------------
def encode_cat (df, cat_col, enc_col):
    gle = LabelEncoder()
    # Fill missing values with a dummy string, otherwise fit_transform throws an error
    #cat_labels = gle.fit_transform(df[cat_col].fillna('Empty'))
    cat_labels = gle.fit_transform(df[cat_col])
    #cat_mappings = {index: label for index, label in enumerate(gle.classes_)}
    df[enc_col] = cat_labels
    
# -----------------------------------------------------
# One-hot encode categorical columns
# -----------------------------------------------------
def encode_cat_onehot (df, cat_col):
    # Drop the first encoded value to avoid the "dummy variable trap"
    # NB: You can also use sklearn's OneHotEncoder() but that returns a numpy array
    # which will have to be converted into a dataframe
    onehot_df = pd.get_dummies(df[cat_col], drop_first=True)
    #onehot_df = pd.get_dummies(df[cat_col].fillna('Empty'), drop_first=True)
    return (pd.concat([df, onehot_df], axis=1))


# -----------------------------------------------------
# Get a list of all the unique values for a feature (usually categorical)
# -----------------------------------------------------
def get_unique_values (df, target):
      return df[target].unique()

# -----------------------------------------------------
# Get a count of unique values for 'Object' columns
# -----------------------------------------------------
def get_unique_counts (df):
      return (df.select_dtypes('object').apply(pd.Series.nunique, axis = 0))

# -----------------------------------------------------
# Print the most frequently occuring values of each categorical column
# -----------------------------------------------------
def count_freq (df, cat_cols):
      for col in cat_cols:
          print (df[col].value_counts().nlargest())

# -----------------------------------------------------
# Visualise basic stats of the data
# -----------------------------------------------------
def data_stats (self):
    df = self.df
    p = Plotter()
    
    # Get all the datatypes
    dd = self.get_dtypes (df)
    p.create_subplot ('count', dd, 'Column Type')
    
    # Unique value counts
    ud = self.get_unique_counts(df)
    p.create_subplot ('series', ud, 'dummy')

    # Missing value counts
    miss_val = self.missing_values_summary(df)
    cols = self.get_feature_names(miss_val)
    p.plot_grid ('multicol', miss_val, cols)

In [ ]:
# VISUALISATION PLOTTING
# ===============================================================

def plot_corr(df):
    # Heatmap of correlations
    plt.figure(figsize = (8, 6))
    sns.heatmap(df.corr(), vmin = -0.25, annot = True, vmax = 0.6, cmap = plt.cm.RdYlBu_r, fmt='.2f', linewidths=.05)
    plt.title('Correlation Heatmap');

# BIVARIATE VISUALISATION
# ===============================================================

# -----------------------------------------------------
# Plots bar plots between a feature and a target pair
# -----------------------------------------------------
def plot_pair_target (df, features, target):
    grid_size = 4
    fig = plt.figure(figsize = (15, 15))
    plt.subplots_adjust(top=.85, hspace=0.3, wspace=0.3)
    
    for i, feature in enumerate(features):
        ax = fig.add_subplot(grid_size, grid_size, i + 1)
        sns.barplot(x = feature, y = target, data=df, ax = ax)
        ax.set_xlabel("Values")
        ax.set_ylabel("Counts") 
        ax.set_title('{} Totals'.format(feature), fontsize=16)

# -----------------------------------------------------
# Plot Stacked histograms of each feature vs target
# -----------------------------------------------------
def plot_stackhist(il, df, features, target):
    grid_size = 4
    fig = plt.figure(figsize = (20, 20))
    plt.subplots_adjust(top=.85, hspace=0.3, wspace=0.3)
    
    # Get list of unique values of the target
    target_values = il.get_unique_values (df, target)
        
    for i, feature in enumerate(features):
        ax = fig.add_subplot(grid_size, grid_size, i + 1)
        # For each value of the target
        dist = [df[df[target]==val][feature] for val in target_values]
        ax.hist(x = dist, stacked=True, label = target_values)
        ax.set_xlabel("Values")
        ax.set_ylabel("Counts") 
        ax.set_title('{} Distribution'.format(feature), fontsize=16)
        ax.legend()

### Image Classification

In [ ]:
#----------------------------------------------------
# Image Classification data preparation pipeline
#----------------------------------------------------
class ImageClassificationDataBundle(DataBundle):
  def __init__(self, folder_path):
    print ('--------- Image Classification DataBundle init', folder_path)

    # Get list of image file extensions
    # Load all files with image extensions from the given folder path
    # Split based on training and validation sub-folders in the top-level folder path
    # Loaded files can be directly used as 'x' items, no extraction necessary
    # Use the parent folder name of each file as the class name for the 'y' labels. Done through a custom extraction function.
    # Convert the 'x' items from Image Files to Images
    # Convert the 'y' items from Class Names to Class Ids
    # At runtime, dynamically read an Image and apply some image processing steps. Finally
    # convert to tensors of floats

    image_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))
    ' '.join(image_extensions)
    load_params = {'source': FolderItemContainer, 'target_cls': ImageFileItemList, 'folder_path': folder_path, 'extensions': image_extensions}
    split_params = {'split_procedure': 'split_path', 'train_folder': 'train', 'valid_folder': 'val'}
    extract_x_params = {'extract_procedure': None, 'target_cls': None}
    y_custom_fn=partial(FileNameItemList.get_ancestor, up_level=1)
    extract_y_params = {'extract_procedure': 'extract_custom', 'target_cls': ClassNameItemList, 'custom_fn': y_custom_fn}
    convert_x_params = [
        {'target_cls': ImageItemList, 'convert_procedure': 'FileToImage'}, 
    ]
    convert_y_params = [
        {'target_cls': ClassIdItemList, 'convert_procedure': 'NameToId'} 
    ]
    xform_x_params = [
        {'xform_procedure': 'make_rgb'}, 
        {'xform_procedure': 'resize', 'size': 128},
        {'xform_procedure': 'to_byte_tensor'},        
        {'xform_procedure': 'to_float_tensor'},
        {'xform_procedure': 'normalise'},
        # Example only {'xform_procedure': 'Custom', 'custom_fn': func}        
    ]
    # These dl_params are not really required for this DataBundle. They are there as test cases for different param combinations
    dl_params = (
        {'bs': 6, 'sampler_fn': RandomSampler},     # for training
        {'bs': 6, 'sampler_fn': SequentialSampler}  # for valid/test
    )
    self.post_proc_params = [
        {'proc_procedure': 'set_mean_std', 'mean': [0.5, 0.5, 0.5], 'std':[0.5, 0.5, 0.5]}, 
    ]
    self.display_params = {
        'layout_procedure': 'display_images', 'figsize': (20, 5),
        'xyz_procedures': ('image', 'label', 'label')
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=xform_x_params, dl_params=dl_params)

In [ ]:
indb = ImageClassificationDataBundle(path_imagenette)
indb.process()

In [ ]:
# Image Classification class names
indb.valid_ds.y.class_vocab_i2n

In [ ]:
indb.display_batch()

In [ ]:
inex_idxs=[10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 23, 24, 42, 43, 44]
indb.display_batch(idxs=inex_idxs, num_cols=5, figsize=(25, 15))

In [ ]:
indb.train_ds.x.add_xform([
  {'xform_procedure': 'make_rgb'},
  {'xform_procedure': 'resize', 'size': 128},
  {'xform_procedure': 'random_resized_crop', 'crop_size': 128},
  {'xform_procedure': 'flip_rotate'},
  {'xform_procedure': 'perspective_warp', 'crop_size': 128},
  {'xform_procedure': 'aug', 'aug_name': 'Random Brightness Contrast'},
  {'xform_procedure': 'aug', 'aug_name': 'RGB Shift'},
  {'xform_procedure': 'to_byte_tensor'},
  {'xform_procedure': 'to_float_tensor'},
  {'xform_procedure': 'normalise'},
])
indb.train_ds.x.xforms

In [ ]:
indb.display_batch(idxs=inex_idxs, num_cols=5, figsize=(25, 15))

In [ ]:
assert(indb.train_ds.x.pair_type is None)

In [ ]:
indb = ImageClassificationDataBundle(path_imagenette)
indb.process()

In [ ]:
num_objs = 4
x_objs = [torch.rand((3, 128, 128)) for _ in range(num_objs)]
y_objs = [torch.randint(5, ()) for _ in range(num_objs)]
z_objs = [torch.randint(5, ()) for _ in range(num_objs)]

indb.display_results(x_objs, y_objs, z_objs, idxs=[0, 1, 2, 3])

### Text Classification CSV

In [ ]:
#----------------------------------------------------
# Text Classification from CSV data preparation pipeline
#----------------------------------------------------
class TextClassificationCSVDataBundle(DataBundle):
  def __init__(self, csv_path):
    print ('--------- Text Classification DataBundle init', csv_path)

    # Load all rows from the given CSV file
    # Split randomly based on a percentage ratio for training and validation
    # 'x' items are taken from 'Reviews' column as text sentences and
    # 'y' labels are taken from 'Sentiments' column as class name labels
    # Convert the 'x' items from Sentences to Words to Word Ids
    # Convert the 'y' items from Class Names to Class Ids

    load_params = {'source': CSVItemContainer, 'target_cls': DfItemList, 'csv_path': csv_path}
    split_params = {'split_procedure': 'split_random', 'train_ratio': 0.8, 'valid_ratio': 0.2}
    extract_x_params = {'extract_procedure': 'extract_colval', 'target_cls': SentenceItemList, 'col': 'reviews'}
    extract_y_params = {'extract_procedure': 'extract_colval', 'target_cls': ClassNameItemList, 'col': 'sentiments'}
    convert_x_params = [
        {'target_cls': SentenceWordItemList, 'convert_procedure': 'SentenceToWord'},
        {'target_cls': SentenceWordIdItemList, 'convert_procedure': 'WordToWordId'}
    ]
    convert_y_params = [
        {'target_cls': ClassIdItemList, 'convert_procedure': 'NameToId'} 
    ]
    # These dl_params are not really required for this DataBundle. They are there as test cases for different param combinations
    dl_params = (
        {'bs': 2, 'shuffle': True},     # for training
        {'bs': 4, 'shuffle': False}     # for valid/test
    )
    self.display_params = {
        'layout_procedure': 'display_texts'
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, dl_params=dl_params)

In [ ]:
tcdb = TextClassificationCSVDataBundle('tc.csv')
tcdb.process()
tcdb.all_rows[:2], len(tcdb.all_rows)

In [ ]:
# CSV Text Classification vocab and class names
tcdb.train_ds.x.vocab_i2w, tcdb.train_ds.y.class_vocab_i2n

In [ ]:
tcdb.display_batch(idxs=[0, 2, 3])

### Language Model

In [ ]:
#----------------------------------------------------
# Language Model from folder data preparation pipeline
#----------------------------------------------------
class LanguageModelFolderDataBundle(DataBundle):
  def __init__(self, folder_path, ds_params):
    print ('--------- IMDB Language Model DataBundle init', folder_path, ds_params)

    # Load all files with text extensions from specific sub-folders within the given folder path
    # Split randomly based on a percentage ratio for training and validation
    # 'x' items are taken by reading the entire document from each file as a single text sentence and
    # 'y' labels are dummy values which will be ignored. Actual Language Model 'y' labels will be generated during training.
    # Convert the 'x' items from Sentences to Words to Word Ids
    # No conversion needed for the dummy 'y' items

    # !!!!!!!!!!!! Split random ratios kept temporarily low to speed up text processing

    txt_extensions = '.txt'
    lm_pre_rules = ["fixup_text", "replace_rep", "replace_wrep", "spec_add_spaces", "rm_useless_spaces", "sub_br"]
    load_params = {'source': FolderItemContainer, 'target_cls': TextFileItemList, 'folder_path': folder_path, 'include_subfolders': ['train', 'test', 'unsup'], 'extensions': txt_extensions}
    split_params = {'split_procedure': 'split_random', 'train_ratio': 0.03, 'valid_ratio': 0.02}
    extract_x_params = {'extract_procedure': 'extract_doc', 'target_cls': SentenceItemList, 'pre_rules': lm_pre_rules}
    extract_y_params = {'extract_procedure': 'extract_dummy', 'target_cls': DummyItemList}
    convert_x_params = [
        {'target_cls': SentenceWordItemList, 'convert_procedure': 'SentenceToWord'},
        {'target_cls': SentenceWordIdItemList, 'convert_procedure': 'WordToWordId'},
        {'target_cls': StreamWordIdItemList, 'convert_procedure': 'WordIdToStream'}
    ]
    convert_y_params = [
    ]
    self.display_params = {
        'layout_procedure': 'display_texts'
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, ds_params=ds_params)

In [ ]:
lmds_params = {'target_ds': FastaiLMDataset, 'bs': 64, 'bptt': 70}
lmdb = LanguageModelFolderDataBundle(path_imdb, lmds_params)
lmdb.process()
lm_vocab = lmdb.convert_state_x['vocab_i2w']

In [ ]:
lmdb.display_batch(idxs=[0, 2, 3])

### Text Classification Folder

In [ ]:
#----------------------------------------------------
# Text Classification from Folder data preparation pipeline
# The processing is almost identical to the Language Model from Folder, except that we leave
# the words from each sentence separate rather than create a continuous stream. And that the
# y labels are taken from folder names rather than dummy values 
#----------------------------------------------------
class TextClassificationFolderDataBundle(DataBundle):
  def __init__(self, folder_path, bs, vocab_i2w=None):
    print ('--------- IMDB Classification DataBundle init', folder_path)

    # Take data only from 'train' and 'test' sub-folders
    # Load all files with text extensions from specific sub-folders within the given folder path
    # Split randomly based on a percentage ratio for training and validation
    # 'x' items are taken by reading the entire document from each file as a single text sentence and
    # 'y' labels are taken from folder names.
    # Convert the 'x' items from Sentences to Words to Word Ids
    # 

    # !!!!!!!!!!!! Split random ratios kept temporarily low to speed up text processing

    txt_extensions = '.txt'
    lm_pre_rules = ["fixup_text", "replace_rep", "replace_wrep", "spec_add_spaces", "rm_useless_spaces", "sub_br"]
    load_params = {'source': FolderItemContainer, 'target_cls': TextFileItemList, 'folder_path': folder_path, 'include_subfolders': ['train', 'test'], 'extensions': txt_extensions}
    split_params = {'split_procedure': 'split_random', 'train_ratio': 0.03, 'valid_ratio': 0.02}
    extract_x_params = {'extract_procedure': 'extract_doc', 'target_cls': SentenceItemList, 'pre_rules': lm_pre_rules}

    y_custom_fn=partial(FileNameItemList.get_ancestor, up_level=1)
    extract_y_params = {'extract_procedure': 'extract_custom', 'target_cls': ClassNameItemList, 'custom_fn': y_custom_fn}

    convert_x_params = [
        {'target_cls': SentenceWordItemList, 'convert_procedure': 'SentenceToWord'},
        {'target_cls': SentenceWordIdItemList, 'convert_procedure': 'WordToWordId', 'vocab_i2w': vocab_i2w}
    ]
    convert_y_params = [
        {'target_cls': ClassIdItemList, 'convert_procedure': 'NameToId'} 
    ]
    # We use different samples for training and validation. Also, the custom sampler functions need to take a sort-key-function
    # as an extra argument. The key function needs the 'data' argument pre-bound using a partial.
    len_key_fn = lambda i,data: len(data[i])
    dl_params = (
        {'bs': bs, 'sampler_fn': SortishSampler, 'key_fn': len_key_fn, 'collate_fn': pad_collate},    # for training
        {'bs': bs, 'sampler_fn': SortSampler, 'key_fn': len_key_fn, 'collate_fn': pad_collate}        # for valid/test
    )
    self.display_params = {
        'layout_procedure': 'display_texts'
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, dl_params=dl_params)

  # ----------------------------
  # Since we use Sorting Samplers which require a key function (which also requires a partial wrapper)
  # we have to override the parent DataBundle with a custom get_sampler() 
  # ----------------------------
  def get_sampler(self, ds, in_train, bs, sampler_fn, key_fn, **kwargs):
    key=partial(key_fn, data=ds.x)
    # The two sampler functions take different arguments. Since we know the sampler
    # functions we can hardcode their names. The sampler_fn that is passed in is the
    # same, but we ignore it.
    if (in_train):
      sampler = SortishSampler(ds.x, key=key, bs=bs)
    else:
      sampler = SortSampler(ds.x, key=key)
    return sampler


In [ ]:
tlmdb = TextClassificationFolderDataBundle(path_imdb, bs=4, vocab_i2w=lm_vocab)
tlmdb.process()

In [ ]:
# IMDB Text Classification vocab
tlmdb.train_ds.x.vocab_i2w[30:40]

### Image Segmentation Image with Mask

In [ ]:
g_data_path = Path(gd_path) / 'data'
subset_masks_path = g_data_path/'train_masks_subset'
subset_imgs_path = g_data_path/'train_subset'

masks_path = subset_masks_path
imgs_path = subset_imgs_path
masks_file_path = subset_imgs_path/'train_masks.csv'

In [ ]:
def mask_name_fn(row):
  col = 'img'
  mn = f'{row[col][:-4]}_mask.gif'
  return mn

#----------------------------------------------------
# Image Segmentation preparation pipeline
#----------------------------------------------------
class ImageSegmentationDataBundle(DataBundle):
  def __init__(self, csv_path, img_folder_path, mask_folder_path):
    print ('--------- Image Segmentation DataBundle init', csv_path, img_folder_path, mask_folder_path)

    # Load all rows from the given CSV file
    # Split sequentially. based on a percentage ratio for training and validation. 
    #   We do this sequentially rather than randomly because each car is in a set of 16 
    #   images taken from different angles. And for a particular car we don't want some
    #   of those images to be in the training set and some in the validation set. Otherwise
    #   it will validate on the same car on which it got trained and will give artificially
    #   good validation results. To address this we make sure that the entire set of images
    #   for a particular car is entirely in training or entirely in validation. Since the
    #   items in our source CSV file are sorted and grouped by car, the set of images for a particular
    #   car appear sequentially in that file. So when we split sequentially we are ensuring that the
    #   first 'n' car sets are all in training and the next 'm' car sets are all in validation.
    # 'x' items are taken from 'img' column as image file names and
    # 'y' labels are taken from 'img' column and then transformed into mask file names
    # Convert the 'x' items from Image Files to Images
    # Convert the 'y' items from Image Files to Images
    # At runtime, dynamically read an Image and apply some image processing steps. Finally
    # convert to tensors of floats

    load_params = {'source': CSVItemContainer, 'target_cls': DfItemList, 'csv_path': csv_path}
    split_params = {'split_procedure': 'split_sequential', 'train_ratio': 0.8, 'valid_ratio': 0.2}
    extract_x_params = {'extract_procedure': 'extract_colval', 'target_cls': ImageFileItemList, 'col': 'img', 'folder_path': img_folder_path}
    extract_y_params = {'extract_procedure': 'extract_custom', 'target_cls': ImageFileItemList, 'folder_path': mask_folder_path, 'custom_fn': mask_name_fn}
    convert_x_params = [
        {'target_cls': ImageItemList, 'convert_procedure': 'FileToImage', 'pair_type': 'mask'}, 
    ]
    convert_y_params = [
        {'target_cls': ImageItemList, 'convert_procedure': 'FileToImage'}, 
    ]
    xform_x_params = [
        {'xform_procedure': 'make_rgb'}, 
        {'xform_procedure': 'resize', 'size': 100},
        {'xform_procedure': 'perspective_warp', 'crop_size': 100},
        {'xform_procedure': 'aug', 'aug_name': 'Horizontal Flip'},
        {'xform_procedure': 'aug', 'aug_name': 'Rotate'},
        {'xform_procedure': 'aug', 'aug_name': 'Random Brightness Contrast'},
        {'xform_procedure': 'to_byte_tensor'},
        {'xform_procedure': 'to_float_tensor'},
        {'xform_procedure': 'normalise'},
        #{'xform_procedure': 'resize', 'size': 128},
        #{'xform_procedure': 'to_byte_tensor'},        
        #{'xform_procedure': 'to_float_tensor'}
    ]
    ds_params = {'target_ds': ILPairedDataset}
    dl_params = (
        {'bs': 6, 'sampler_fn': RandomSampler},     # for training
        {'bs': 6, 'sampler_fn': SequentialSampler}  # for valid/test
    )
    self.post_proc_params = [
        {'proc_procedure': 'set_mean_std', 'mean': [0.485, 0.456, 0.406], 'std':[0.229, 0.224, 0.225]}, 
    ]
    self.display_params = {
        'layout_procedure': 'display_images', 'figsize': (20, 5),
        'xyz_procedures': ('image', 'mask', 'label')
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=xform_x_params, ds_params=ds_params, dl_params=dl_params)

In [ ]:
isdb = ImageSegmentationDataBundle(masks_file_path, imgs_path, masks_path)
isdb.process()

In [ ]:
isdb.display_batch()

In [ ]:
assert(isdb.train_ds.x.pair_type == 'mask')
assert(isdb.valid_ds.x.pair_type == 'mask')
assert(isdb.train_ds.y.pair_type is None)
assert(isdb.valid_ds.y.pair_type is None)

### Cycle GAN Tuple Image List

In [ ]:
#export

#----------------------------------------------------
# Image Cycle GAN data preparation pipeline
# In general, it is very similar to an Image Classification pipeline except that it provides pairs of images (ie. a tuple)
# for each data row instead of a single image
#----------------------------------------------------
class ImageCycleGanDataBundle(DataBundle):
  def __init__(self, folder_path, multi_path):
    print ('--------- Image Cycle GAN DataBundle init', folder_path)

    # Get list of image file extensions
    # Under the given folder path, image files in each sub-folder in 'multi_path' are loaded into separate sub-lists.
    # No Split is required as we don't need a validation set
    # Loaded files can be directly used as 'x' items, no extraction necessary
    # 'y' labels are dummy values which will be ignored. No labels are needed during training.
    # Convert the 'x' items from Image Files to tuples of Images. A tuple contains one image from each sub-list.
    # No conversion needed for the dummy 'y' items
    # At runtime, dynamically read an Image and apply some image processing steps. Finally
    # convert to tensors of floats. These image processing steps are identical to the ones for single
    # images, except that they operate on tuples of images.

    image_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))
    ' '.join(image_extensions)
    load_params = {'source': FolderMultiItemContainer, 'target_cls': FileMultiItemList, 'folder_path': folder_path, 'extensions': image_extensions, 'multi_path': multi_path}
    split_params = {'split_procedure': None}
    extract_x_params = {'extract_procedure': None, 'target_cls': None}
    extract_y_params = {'extract_procedure': 'extract_dummy', 'target_cls': DummyItemList}
    convert_x_params = [
        {'target_cls': ImageTupleItemList, 'convert_procedure': 'FileMultiToImageTuple'}, 
    ]
    convert_y_params = [
    ]
    xform_x_params = [
        {'xform_procedure': 'tuple_make_rgb'}, 
        {'xform_procedure': 'tuple_resize', 'size': 128},
        {'xform_procedure': 'tuple_to_byte_tensor'},        
        {'xform_procedure': 'tuple_to_float_tensor'}
     ]
    # These dl_params are not really required for this DataBundle. They are there as test cases for different param combinations
    dl_params = (
        {'bs': 1, 'sampler_fn': RandomSampler},     # for training
        {'bs': 1, 'sampler_fn': SequentialSampler}  # for valid/test
    )
    self.display_params = {
        'layout_procedure': 'display_images', 'figsize': (20, 5),
        'xyz_procedures': ('image', 'label', 'image')
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=xform_x_params, dl_params=dl_params)

#----------------------------------------------------
# A Folder Multi-Item is similar to a Folder container, except that it stores files in different
# sub-folders in separate sub-lists. 'multi_path' defines the list of sub-folders directly under the
# root 'folder_path'. One sub-list is created for each 'multi_path' sub-folder with all the files in
# that sub-folder.
#----------------------------------------------------
class FolderMultiItemContainer(ItemContainer):
  def __init__(self, folder_path, multi_path, extensions=None):
    # NB: For now, we support only two paths
    assert (isinstance(multi_path, list) and (len(multi_path) == 2))
    self.folder_path = Path(folder_path)
    self.extensions = extensions
    # 'A' and 'B' sub-folders
    self.A_path, self.B_path = multi_path[0], multi_path[1]
    super().__init__()

  # ----------------------------
  # Use the FolderItemContainer to load files in each sub-folder
  # We create A and B sub-lists since we support only two sub-folders
  # ----------------------------
  def load(self, target_cls):
    # Create 'A' and 'B' FolderItemContainers, one for each sub-folder
    A_cntnr = FolderItemContainer(self.folder_path, include_subfolders=[self.A_path], extensions=self.extensions)
    B_cntnr = FolderItemContainer(self.folder_path, include_subfolders=[self.B_path], extensions=self.extensions)

    # Load the 'A' and 'B' sub-folders into 'A' and 'B' sub-lists
    A_file_list = A_cntnr.load(ImageFileItemList)[:]
    B_file_list = B_cntnr.load(ImageFileItemList)[:]

    # !!!!!!!!!! Remove this now. 'A' and 'B' sub-lists
    #A_file_list = A_cntnr.all_rows[:]
    #B_file_list = B_cntnr.all_rows[:]

    # Create the Item List with the list of files
    all_rows = target_cls([A_file_list, B_file_list])
    return all_rows

#----------------------------------------------------
# Similar to a FileItemList except that it stores multiple separate lists of files
# It doesn't do much, except a 'pass-through' conversion to an Image Tuple item list
#----------------------------------------------------
class FileMultiItemList(ItemList):
  def FileMultiToImageTuple(self, items, convert_state, in_train, **kwargs):
    filemulti_items = items
    # 'filemulti_items' contains two sub-lists. Pass them through as is to the
    # Image Tuple item list.
    image_tuple_items = filemulti_items
    return (image_tuple_items)

  def extract_dummy(self, **kwargs):
    y_list = [0] * len(self.items[0])
    return (y_list)

#----------------------------------------------------
# Similar to an ImageItemList except that each item is a pair of images. It stores
# two sub-lists of images, which are of different lengths. When asked for the i-th
# item, it gets the i-th image from the first (ie. 'A') sub-list, and then fetches one
# image at random from the second (ie. 'B') sub-list.
#----------------------------------------------------
class ImageTupleItemList(ImageItemList):
  def __init__(self, image_tuple_items, **kwargs):
    assert(len(image_tuple_items) == 2)
    assert(isinstance(image_tuple_items[0], list) and isinstance(image_tuple_items[1], list))
    # 'image_tuple_items' contains two sub-lists
    items, self.B_items = image_tuple_items
    super().__init__(items, **kwargs)

  # ----------------------------
  # For displaying items, we combine the pair of images into a single stacked image
  # ----------------------------
  def get_display_data(self, obj, idx):
    img_tensor_tuple = obj
    assert(isinstance(img_tensor_tuple[0], torch.Tensor))
    # dim=1 will stack the two images vertically. To do it horizontally, use dim=2
    merged = torch.cat(img_tensor_tuple, dim=1)
    return (merged)

  # ----------------------------
  # Get a pair of images rather than a single image
  # ----------------------------
  def _get_object(self, item):
    # Let the parent Image list load the image for an image file item.
    obj = super()._get_object(item)

    # Fetch another image file at random from the 'B' list and let the parent load the 
    # image for that as well.
    B_obj = super()._get_object(self.B_items[random.randint(0, len(self.B_items)-1)])

    # Return the pair
    return ((obj, B_obj))

  # ----------------------------
  # Support all the same image xform transforms as an Image item list except that
  # we operate on a pair of images. Rather than re-implementing them, we leverage the parent object's
  # transforms by calling them twice for the pair of images.
  #
  # The xform methods on the child start with a prefix of 'tuple_' but are otherwise identical 
  # to the parent's. When any of the child's 'tuple_xform_*' methods is called, with a pair of
  # images as input, __getattr_() gets invoked. The parent's method gets bound into a wrapper
  # function which we return here.
  #
  # Python then calls the wrapper function passing it the pair of images as input. The wrapper
  # in turn calls the parent's method for each of the images in the pair, and then returns
  # the result
  # ----------------------------
  def __getattr__(self, attr):
    # Strip off the 'tuple_' prefix to get the name of the parent method
    # We don't invoke the parent's method here, but invoke it from the wrapper
    # function instead. Since the wrapper is a nested function the 'super_method' 
    # gets bound into it.
    super_attr = attr.partition('tuple_')[2]
    super_method = getattr(super(), super_attr, None)
    #
    # Dispatching based on passed arguments is a two-step process:
    #    __getattr__ returns a proxy wrapper method
    #    python calls the proxy wrapper which then decides which real method to call
    def _tuple_to_image_wrapper(tuple_obj, *args, **kwargs):
      A_obj, B_obj = tuple_obj

      # The parent's 'super_method' gets bound inside the wrapper
      # Call the parent's method for the 'A' and 'B' images
      A_res = super_method(A_obj, *args, **kwargs)
      B_res = super_method(B_obj, *args, **kwargs)

      # Return the result as a pair
      return ((A_res, B_res))

    return _tuple_to_image_wrapper

In [ ]:
icgdb = ImageCycleGanDataBundle(path_imagenette, multi_path=['train', 'val'])
icgdb.process()

icgdb.display_batch()

In [ ]:
icgdb.display_batch(idxs=[49])

icg_x, icg_y = next(iter(icgdb.train_dl))
type(icg_x), icg_x[0].shape, icg_x[1].shape, type(icg_y), icg_y.shape, icg_y

### Audio Classification

In [ ]:
#export

def re_labeler(fn, pat): return re.findall(pat, str(fn))[0]
label_pat = r'/([mf]\d+)_'
speaker_labeler = partial(re_labeler, pat=label_pat)

#----------------------------------------------------
# Audio Classification data preparation pipeline
#----------------------------------------------------
class AudioClassificationDataBundle(DataBundle):
  def __init__(self, folder_path):
    print ('--------- Audio Classification DataBundle init', folder_path)

    # Get list of audio file extensions
    # Load all files with audio extensions from the given folder path
    # Split randomly based on a percentage ratio for training and validation
    # Loaded files can be directly used as 'x' items, no extraction necessary
    # Use a substring from the file name of each file as the class name for the 'y' labels. Done through a custom extraction function.
    # Convert the 'x' items from Audio Files to Audio
    # Convert the 'y' items from Class Names to Class Ids
    # At runtime, dynamically read an Audio File and apply some pre-processing steps to create spectrograms

    audio_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('audio/'))
    ' '.join(audio_extensions)
    load_params = {'source': FolderItemContainer, 'target_cls': AudioFileItemList, 'folder_path': folder_path, 'extensions': audio_extensions}
    split_params = {'split_procedure': 'split_random', 'train_ratio': 0.8, 'valid_ratio': 0.2}
    extract_x_params = {'extract_procedure': None, 'target_cls': None}
    extract_y_params = {'extract_procedure': 'extract_custom', 'target_cls': ClassNameItemList, 'custom_fn': speaker_labeler}
    convert_x_params = [
        {'target_cls': AudioItemList, 'convert_procedure': 'FileToAudio'}, 
    ]
    convert_y_params = [
        {'target_cls': ClassIdItemList, 'convert_procedure': 'NameToId'} 
    ]
    xform_x_params = [
        {'xform_procedure': 'pad_trim', 'max_ms': 3000},
        {'xform_procedure': 'signal_shift', 'max_shift_pct': 0.4},
        {'xform_procedure': 'spectro_gram'},
        {'xform_procedure': 'spectro_augment', 'max_mask_pct': 0.1, 'n_freq_masks': 2, 'n_time_masks': 2},
    ]
    # These dl_params are not really required for this DataBundle. They are there as test cases for different param combinations
    dl_params = (
        {'bs': 6, 'sampler_fn': RandomSampler},     # for training
        {'bs': 6, 'sampler_fn': SequentialSampler}  # for valid/test
    )
    self.display_params = {
        'layout_procedure': 'display_audio', 'figsize': (30, 6),
    }
    super().__init__(load_params, split_params, extract_x_params, extract_y_params, convert_x_params, convert_y_params, xform_x_params=xform_x_params, dl_params=dl_params)

**Download data**

In [ ]:
audio_path=Path.cwd()/'audio'
dsid = "ST-AEDS-20180100_1-OS"
!wget 'http://www.openslr.org/resources/45/{dsid}.tgz'

import tarfile
tf = tarfile.open(f'{dsid}.tgz')
tf.extractall(audio_path)
list(audio_path.iterdir())[:3]

In [ ]:
acdb = AudioClassificationDataBundle(audio_path)
acdb.process()

In [ ]:
file_orig = acdb.train_ds.x.items[0]
sample_orig = AudioUtil.open(file_orig)
AudioUtil.play(sample_orig)

sample_aud = acdb.train_ds.x[0]
AudioUtil.show_spectro(sample_aud)

In [ ]:
# Audio Classification class names
acdb.valid_ds.y.class_vocab_i2n

In [ ]:
acdb.display_batch()

### Export

In [ ]:
!wget https://raw.githubusercontent.com/ketanhdoshi/ml/master/lib/nb_export.py

In [ ]:
from nb_export import notebook2scriptSingle
notebook2scriptSingle(gn_path + '/lib/data_lib.ipynb', gn_path + '/exp')

### Old DfItemList code

In [ ]:


    len_df = len(df.index)
    if (items is None):
      # Items should be a range of consecutive numbers from 0 to len_df - 1 
      items = list(range(len_df))
    elif (len(items) == len_df):
      # Most likely, items already contains the required numbers, but we re-create
      # it just to be sure
      items = list(range(len_df))
    elif (len(items) < len_df):
      # Items contains the indexes of a subset of the Dataframe. This happens when we
      # split the ItemList. Create a new Dataframe with those indexes
      df = df.iloc[items, :].reset_index(drop=True)
      new_len_df = len(df.index)
      items = list(range(new_len_df))
    else:
      # Items is longer than the Dataframe. This should never happen.
      assert (False)


  # ----------------------------
  # Create a new dataframe with just the given subset of rows ie. 'items'
  # ----------------------------
  def WAIT_new(self, items):
    if (items is None):
      return (None)

    # Make a new dataframe by selecting the given 'items' and create a new
    # ItemList from it
    new_df = self.df.iloc[items, :].reset_index(drop=True)

    # Keyword arguments for initialisation using the attributes to be copied over 
    # to the cloned ItemList
    copy_args = {k:getattr(self, k, None) for k in self._copy_new}
    # Add the list of items to the keyword arguments
    kwargs = {'df': new_df, **copy_args}
    # Now create a new instance of my ItemList type using the keyword arguments
    il = self.__class__(**kwargs)

    return (il)


### Old Test the image data bundle

In [ ]:
indb = ImageClassificationDataBundle(path_imagenette)
indb.process()

In [ ]:
my_item = indb.train_ds.x._get_item_for_split_wo_xform(img_idx)
my_img = indb.train_ds.x._get_object(my_item)

In [ ]:
img_idx=55
foo=indb.train_ds.x[img_idx]
foo_y=indb.train_ds.y[img_idx]
print('class label is', indb.convert_state_y['class_vocab_i2n'][foo_y])
foo.type(), foo.shape, indb.convert_state_y
show_image(foo)

### Old Show Item/Batch

In [ ]:

  # ----------------------------
  # Dynamically apply any defined transforms on an item and return the resulting object
  # ----------------------------
  def _WAIT_xform(self, item):
    # Get the object for this item 
    obj = self._get_object(item)

    # Apply transforms on that object. The output of each transform becomes the input
    # for the next transform
    if (self.xforms is not None):
      for xform in self.xforms:
        obj = self._do_xform(obj, **xform)

    return obj

  # ----------------------------
  # Built-in dunder method to get an object or a list of objects from the ItemList, using
  # standard indexing and slicing. So 'idx' can be either a single index, a range or a slice
  # We get the data object, apply transforms on it and return the result
  # ----------------------------
  def WAIT__getitem__(self, idx):
    # Get the item (or list of items) corresponding to the given indices
    item = super().__getitem__(idx)

    if isinstance(item,list):
      # Return a list of objects after applying transforms on each one
      return [self._xform(o) for o in item]
    else:
      # Return a single object after applying transforms on it
      return self._xform(item)


In [ ]:
def show_image(im, figsize=(3,3)):
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(im.permute(1,2,0))

def show_batch(x, c=4, r=None, figsize=None):
    n = len(x)
    if r is None: r = int(math.ceil(n/c))
    if figsize is None: figsize=(c*3,r*3)
    fig,axes = plt.subplots(r,c, figsize=figsize)
    for xi,ax in zip(x,axes.flat): show_image(xi, ax)

def show_item(idx, obj_x, obj_y, figsize=(3,3), **kwargs):
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(obj_x.permute(1,2,0))

def show_image(im, ax=None, figsize=(3,3)):
    if ax is None: _,ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(im.permute(1,2,0))

def show_batch(x, c=10, r=None, figsize=None):
    n = len(x)
    if r is None: r = int(math.ceil(n/c))
    if figsize is None: figsize=(c*3,r*3)
    fig,axes = plt.subplots(r,c, figsize=figsize)
    for xi,ax in zip(x,axes.flat): 
      show_image(xi, ax)

def show_batch_transpose (my_img):
  _, axs = plt.subplots(2, 4, figsize=(12, 6))
  for i,ax in enumerate(axs.flatten()):
    if i==0: ax.imshow(my_img)
    else:    ax.imshow(my_img.transpose(i-1))
    ax.axis('off')


#foo_imgs, _ = next(iter(indb.train_dl))
foo_img_tensors = indb.train_ds.x[:15]
foo_img_labels = indb.train_ds.y[:15]
if (False):
  show_item(None, foo, None)
  show_item(None, foo_imgs[0], None)
  show_batch(foo_imgs)

sd=ShowData()
# Show images as Pytorch tensors, after applying transforms
sd.show_images(foo_img_tensors, foo_img_labels)

foo_img_files = indb.train_ds.x.items[:15]
foo_imgs = [indb.train_ds.x._get_object(f) for f in foo_img_files]
# Show images as PIL Image objects
sd.show_images(foo_imgs, foo_img_labels)

### Create a DataLoader using a Sampler and Collate Function

In [ ]:
tcdb.convert_state

In [ ]:
fdl = tcdb.train_dl
idl = iter(fdl)
fdx, fdy = next(idl)
tcdb.train_ds.x, tcdb.train_ds.y, fdx, fdy

In [ ]:
fdl = DataLoader(tcdb.train_ds, batch_size=2)
idl = iter(fdl)
fdx, fdy = next(idl)
tcdb.train_ds.x, tcdb.train_ds.y, fdx, fdy

In [ ]:
fdl = DataLoader(tcdb.train_ds, batch_size=2)
idl = iter(fdl)
fdx, fdy = next(idl)
tcdb.train_ds.x, tcdb.train_ds.y, fdx, fdy

In [ ]:
ss = SortSampler(tcdb.train_ds.x, key=lambda t: len(tcdb.train_ds.x[t]))
sdl = DataLoader(tcdb.train_ds, batch_size=2, sampler=ss)
sidl = iter(sdl)
sfdx, sfdy = next(sidl)
sfdx, sfdy

In [ ]:
sos = SortishSampler(tlmdb.train_ds.x, key=lambda t: len(tlmdb.train_ds.x[t]), bs=10)
sodl = DataLoader(tlmdb.train_ds, batch_size=10, sampler=sos, collate_fn=pad_collate)
soidl = iter(sodl)
sofdx, sofdy = next(soidl)
sofdx, sofdy

In [ ]:
#type(foo), type(indb.train_ds.x[img_idx])
type(indb.train_ds.x.items[img_idx])